### Scrips for generating splits
This script assums you have the main ReaSCAN generated by the generate_ReaSCAN.py script. After that, you can use this file to generate/extrapolate different splits. In the future, we may consolidate two files.

In [1]:
from collections import namedtuple, OrderedDict
import os
from typing import List
from typing import Tuple
import logging
from collections import defaultdict
from collections import Counter
import json
import torch
import numpy as np

def isnotebook():
    try:
        shell = get_ipython().__class__.__name__
        if shell == 'ZMQInteractiveShell':
            return True   # Jupyter notebook or qtconsole
        elif shell == 'TerminalInteractiveShell':
            return False  # Terminal running IPython
        else:
            return False  # Other type (?)
    except NameError:
        return False      # Probably standard Python interpreter
if isnotebook():
    device = torch.device("cpu")
else:
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

FORMAT = "%(asctime)-15s %(message)s"
logging.basicConfig(format=FORMAT, level=logging.INFO,
                    datefmt="%Y-%m-%d %H:%M")
logger = logging.getLogger(__name__)

from world import *
from object_vocabulary import *
from vocabulary import *
from grammer import *
from simulator import *
from relation_graph import *

#### P1: gSCAN Pattern

In [ ]:
p1_path_to_data = "../../data-files-updated/ReaSCAN-compositional-p1/data-train.txt"
logger.info(f"Reading dataset from file: {p1_path_to_data}...")
p1_data_json = json.load(open(p1_path_to_data, "r"))

p1_all_fake_train = p1_data_json["examples"]["train"]
# for dev and test, it is simple, let us just shuffle, and random select.
len(p1_all_fake_train)

In [ ]:
# For generating the splits, we actually have to go through compositional splits first
# and then consider random splits like dev and test. Because, we don't want things mixed up
# in the dev and test. Dev and test should only contain commands that appear in the train,
# so a total random partition at the end should work.

# We do the splits step-by-step!
p1_id_example_map = OrderedDict({})
p1_id_splits_map = OrderedDict({})
index = 0
for example in p1_data_json["examples"]["train"]:
    p1_id_example_map[index] = example
    p1_id_splits_map[index] = set([]) # set of splits that this example belongs to.
    index += 1

In [ ]:
p1_splits_distribution = OrderedDict({})
p1_splits_assignment = OrderedDict({})
for index, splits in p1_id_splits_map.items():
    if len(splits) == 0:
        split = "train" # let us split this up later!
        if split in p1_splits_distribution.keys():
            p1_splits_distribution[split] += 1
        else:
            p1_splits_distribution[split] = 1
        
        if split in p1_splits_assignment:
            p1_splits_assignment[split].append(index)
        else:
            p1_splits_assignment[split] = [index]
    else:   
        assert False

# Let us further segment train into dev and test!
gscan_dev_size = int(len(p1_all_fake_train)*0.01)
gscan_test_size = int(len(p1_all_fake_train)*0.052)
p1_all_example_id = p1_splits_assignment["train"]
random.shuffle(p1_all_example_id)
p1_train_example_id = p1_all_example_id[:(-gscan_dev_size-gscan_test_size)]
p1_dev_example_id = p1_all_example_id[(-gscan_dev_size-gscan_test_size):-gscan_dev_size]
p1_test_example_id = p1_all_example_id[-gscan_dev_size:]
p1_splits_assignment["train"] = p1_train_example_id
p1_splits_assignment["dev"] = p1_dev_example_id
p1_splits_assignment["test"] = p1_test_example_id

In [ ]:
for split, all_ids in p1_splits_assignment.items():
    print(f"for {split} split, we have {len(all_ids)} examples.")

In [ ]:
# remake our data file accordingly.
updated_examples = OrderedDict({})
for split, all_ids in p1_splits_assignment.items():
    updated_examples[split] = []
    for _id in all_ids:
        updated_examples[split].append(p1_id_example_map[_id])

In [ ]:
# save it to the disk
p1_data_json["examples"] = updated_examples
with open("../../data-files-updated/ReaSCAN-compositional-p1/data-compositional-splits.txt", "w") as fd:
    json.dump(p1_data_json, fd, indent=4)

#### P2: Single Clause

In [ ]:
p2_path_to_data = "../../data-files-updated/ReaSCAN-compositional-p2/data-train.txt"
logger.info(f"Reading dataset from file: {p2_path_to_data}...")
p2_data_json = json.load(open(p2_path_to_data, "r"))

p2_all_fake_train = p2_data_json["examples"]["train"]
# for dev and test, it is simple, let us just shuffle, and random select.
len(p2_all_fake_train)

In [ ]:
# For generating the splits, we actually have to go through compositional splits first
# and then consider random splits like dev and test. Because, we don't want things mixed up
# in the dev and test. Dev and test should only contain commands that appear in the train,
# so a total random partition at the end should work.

# We do the splits step-by-step!
p2_id_example_map = OrderedDict({})
p2_id_splits_map = OrderedDict({})
index = 0
for example in p2_data_json["examples"]["train"]:
    p2_id_example_map[index] = example
    p2_id_splits_map[index] = set([]) # set of splits that this example belongs to.
    index += 1

In [ ]:
p2_splits_distribution = OrderedDict({})
p2_splits_assignment = OrderedDict({})
for index, splits in p2_id_splits_map.items():
    if len(splits) == 0:
        split = "train" # let us split this up later!
        if split in p2_splits_distribution.keys():
            p2_splits_distribution[split] += 1
        else:
            p2_splits_distribution[split] = 1
        
        if split in p2_splits_assignment:
            p2_splits_assignment[split].append(index)
        else:
            p2_splits_assignment[split] = [index]
    else:   
        assert False

# Let us further segment train into dev and test!
gscan_dev_size = int(len(p2_all_fake_train)*0.01)
gscan_test_size = int(len(p2_all_fake_train)*0.052)
p2_all_example_id = p2_splits_assignment["train"]
random.shuffle(p2_all_example_id)
p2_train_example_id = p2_all_example_id[:(-gscan_dev_size-gscan_test_size)]
p2_dev_example_id = p2_all_example_id[(-gscan_dev_size-gscan_test_size):-gscan_dev_size]
p2_test_example_id = p2_all_example_id[-gscan_dev_size:]
p2_splits_assignment["train"] = p2_train_example_id
p2_splits_assignment["dev"] = p2_dev_example_id
p2_splits_assignment["test"] = p2_test_example_id

In [ ]:
for split, all_ids in p2_splits_assignment.items():
    print(f"for {split} split, we have {len(all_ids)} examples.")

In [ ]:
# remake our data file accordingly.
updated_examples = OrderedDict({})
for split, all_ids in p2_splits_assignment.items():
    updated_examples[split] = []
    for _id in all_ids:
        updated_examples[split].append(p2_id_example_map[_id])

In [ ]:
# save it to the disk
p2_data_json["examples"] = updated_examples
with open("../../data-files-updated/ReaSCAN-compositional-p2/data-compositional-splits.txt", "w") as fd:
    json.dump(p2_data_json, fd, indent=4)

#### P3: Double Clause

In [ ]:
p3_path_to_data = "../../data-files-updated/ReaSCAN-compositional-p3/data-train.txt"
logger.info(f"Reading dataset from file: {p3_path_to_data}...")
p3_data_json = json.load(open(p3_path_to_data, "r"))

In [ ]:
p3_all_fake_train = p3_data_json["examples"]["train"]
# for dev and test, it is simple, let us just shuffle, and random select.
len(p3_all_fake_train)

In [ ]:
# For generating the splits, we actually have to go through compositional splits first
# and then consider random splits like dev and test. Because, we don't want things mixed up
# in the dev and test. Dev and test should only contain commands that appear in the train,
# so a total random partition at the end should work.

# We do the splits step-by-step!
p3_id_example_map = OrderedDict({})
p3_id_splits_map = OrderedDict({})
index = 0
for example in p3_data_json["examples"]["train"]:
    p3_id_example_map[index] = example
    p3_id_splits_map[index] = set([]) # set of splits that this example belongs to.
    index += 1

In [ ]:
p3_splits_distribution = OrderedDict({})
p3_splits_assignment = OrderedDict({})
for index, splits in p3_id_splits_map.items():
    if len(splits) == 0:
        split = "train" # let us split this up later!
        if split in p3_splits_distribution.keys():
            p3_splits_distribution[split] += 1
        else:
            p3_splits_distribution[split] = 1
        
        if split in p3_splits_assignment:
            p3_splits_assignment[split].append(index)
        else:
            p3_splits_assignment[split] = [index]
    else:   
        assert False

# Let us further segment train into dev and test!
gscan_dev_size = int(len(p3_all_fake_train)*0.01)
gscan_test_size = int(len(p3_all_fake_train)*0.052)
p3_all_example_id = p3_splits_assignment["train"]
random.shuffle(p3_all_example_id)
p3_train_example_id = p3_all_example_id[:(-gscan_dev_size-gscan_test_size)]
p3_dev_example_id = p3_all_example_id[(-gscan_dev_size-gscan_test_size):-gscan_dev_size]
p3_test_example_id = p3_all_example_id[-gscan_dev_size:]
p3_splits_assignment["train"] = p3_train_example_id
p3_splits_assignment["dev"] = p3_dev_example_id
p3_splits_assignment["test"] = p3_test_example_id

In [ ]:
for split, all_ids in p3_splits_assignment.items():
    print(f"for {split} split, we have {len(all_ids)} examples.")

In [ ]:
# remake our data file accordingly.
updated_examples = OrderedDict({})
for split, all_ids in p3_splits_assignment.items():
    updated_examples[split] = []
    for _id in all_ids:
        updated_examples[split].append(p3_id_example_map[_id])

In [ ]:
# save it to the disk
p3_data_json["examples"] = updated_examples
with open("../../data-files-updated/ReaSCAN-compositional-p3/data-compositional-splits.txt", "w") as fd:
    json.dump(p3_data_json, fd, indent=4)

#### P3: Double Clause (combing with P3 sharding, this section is shared across all patterns as well!)

In [ ]:
import os
import json
import re
pattern = "p4"
special_condition = ""
if special_condition != "":
    prefix = f"{pattern}-{special_condition}"
else:
    prefix = pattern
sharding_dir = f"../../data-files-{prefix}/"
if pattern == "p3":
    upper_limit = 3375
elif pattern == "p2":
    upper_limit = 2025
elif pattern == "p1":
    upper_limit = 675
elif pattern == "p4":
    upper_limit = 4000

In [ ]:
unique_command = set([])
for subdir, dirs, files in os.walk(sharding_dir):
    if "jobid" in subdir:
        
        # Completeness check!
        logging_file = os.path.join(subdir, "generator.log")
        with open(logging_file) as f:
            content = f.readlines()
        # you may also want to remove whitespace characters like `\n` at the end of each line
        content = [x.strip() for x in content]
        completed = False
        for c in content:
            if "==FINISH==" in c:
                completed = True
                break
        jobid = logging_file.split("/")[-2].split("-")[-1]
        print(f"jobid={jobid}, status=complete={completed}")
        if not completed:
            break
        
        # Uniqueness check!
        data_file_path = os.path.join(subdir, "data-train.txt")
        print(f"scanning for file: {data_file_path}")
        data_file = json.load(open(data_file_path, "r"))
        for example in data_file["examples"]["train"]:
            command_split = re.split(',a,|,the,', example['command'])
            command_mono = ",".join(command_split)
            unique_command.add(command_mono)
assert len(unique_command) > upper_limit

In [ ]:
len(unique_command)

In [ ]:
shared_example_combined = {}
per_command_mono_count = {}
for subdir, dirs, files in os.walk(sharding_dir):
    if "jobid" in subdir:
        data_file_path = os.path.join(subdir, "data-train.txt")
        print(f"Collecting for file: {data_file_path}")
        data_file = json.load(open(data_file_path, "r"))
        for example in data_file["examples"]["train"]:
            command_split = re.split(',a,|,the,', example['command'])
            command_mono = ",".join(command_split)
            if command_mono in per_command_mono_count.keys():
                if per_command_mono_count[command_mono] == 180: # for p4 this may never hit!
                    continue # we are not adding this example since redundant!
                per_command_mono_count[command_mono] += 1
            else:
                per_command_mono_count[command_mono] = 1
            if command_mono in shared_example_combined.keys():
                shared_example_combined[command_mono].append(example)
            else:
                shared_example_combined[command_mono] = [example]

In [ ]:
# write to disk!
import random
shared_examples = []
commands_mono = list(shared_example_combined.keys())
random.shuffle(commands_mono)
for i in range(upper_limit):
    examples_to_include = shared_example_combined[commands_mono[i]]
    for example in examples_to_include:
        shared_examples.append(example)

In [ ]:
data_file["examples"]["train"] = shared_examples

In [ ]:
print(len(shared_examples))

In [ ]:
with open(f"../../data-files-{prefix}/ReaSCAN-compositional-{prefix}/data-train.txt", "w") as fd:
    json.dump(data_file, fd, indent=4)

#### P3-RD: Double Clause with Only Random Distractors (and some contextual distractors, which are also random)

In [ ]:
p3_rd_path_to_data = "../../data-files-updated/ReaSCAN-compositional-p3-rd/data-train.txt"
logger.info(f"Reading dataset from file: {p3_rd_path_to_data}...")
p3_rd_data_json = json.load(open(p3_rd_path_to_data, "r"))

In [ ]:
p3_rd_all_fake_train = p3_rd_data_json["examples"]["train"]
# for dev and test, it is simple, let us just shuffle, and random select.
len(p3_rd_all_fake_train)

In [ ]:
# For generating the splits, we actually have to go through compositional splits first
# and then consider random splits like dev and test. Because, we don't want things mixed up
# in the dev and test. Dev and test should only contain commands that appear in the train,
# so a total random partition at the end should work.

# We do the splits step-by-step!
p3_rd_id_example_map = OrderedDict({})
p3_rd_id_splits_map = OrderedDict({})
index = 0
for example in p3_rd_data_json["examples"]["train"]:
    p3_rd_id_example_map[index] = example
    p3_rd_id_splits_map[index] = set([]) # set of splits that this example belongs to.
    index += 1

In [ ]:
p3_rd_splits_distribution = OrderedDict({})
p3_rd_splits_assignment = OrderedDict({})
for index, splits in p3_rd_id_splits_map.items():
    if len(splits) == 0:
        split = "train" # let us split this up later!
        if split in p3_rd_splits_distribution.keys():
            p3_rd_splits_distribution[split] += 1
        else:
            p3_rd_splits_distribution[split] = 1
        
        if split in p3_rd_splits_assignment:
            p3_rd_splits_assignment[split].append(index)
        else:
            p3_rd_splits_assignment[split] = [index]
    else:   
        assert False

# Let us further segment train into dev and test!
gscan_dev_size = int(len(p3_rd_all_fake_train)*0.01)
gscan_test_size = int(len(p3_rd_all_fake_train)*0.052)
p3_rd_all_example_id = p3_rd_splits_assignment["train"]
random.shuffle(p3_rd_all_example_id)
p3_rd_train_example_id = p3_rd_all_example_id[:(-gscan_dev_size-gscan_test_size)]
p3_rd_dev_example_id = p3_rd_all_example_id[(-gscan_dev_size-gscan_test_size):-gscan_dev_size]
p3_rd_test_example_id = p3_rd_all_example_id[-gscan_dev_size:]
p3_rd_splits_assignment["train"] = p3_rd_train_example_id
p3_rd_splits_assignment["dev"] = p3_rd_dev_example_id
p3_rd_splits_assignment["test"] = p3_rd_test_example_id

In [ ]:
for split, all_ids in p3_rd_splits_assignment.items():
    print(f"for {split} split, we have {len(all_ids)} examples.")

In [ ]:
# remake our data file accordingly.
updated_examples = OrderedDict({})
for split, all_ids in p3_rd_splits_assignment.items():
    updated_examples[split] = []
    for _id in all_ids:
        updated_examples[split].append(p3_rd_id_example_map[_id])

In [ ]:
# save it to the disk
p3_rd_data_json["examples"] = updated_examples
with open("../../data-files-updated/ReaSCAN-compositional-p3-rd/data-compositional-splits.txt", "w") as fd:
    json.dump(p3_rd_data_json, fd, indent=4)

#### P1+P2+P3: Compositional Splits

In [ ]:
import random
import numpy as np
random.seed(42)
np.random.seed(42)

# Combine all of three together
# We downsample it to make it trainable within reasonable time frame!
p1_path_to_data = "../../data-files-updated/ReaSCAN-compositional-p1/data-train.txt"
logger.info(f"Reading dataset from file: {p1_path_to_data}...")
p1_data_json = json.load(open(p1_path_to_data, "r"))

p2_path_to_data = "../../data-files-updated/ReaSCAN-compositional-p2/data-train.txt"
logger.info(f"Reading dataset from file: {p2_path_to_data}...")
p2_data_json = json.load(open(p2_path_to_data, "r"))

In [ ]:
p3_path_to_data = "../../data-files-updated/ReaSCAN-compositional-p3/data-train.txt"
logger.info(f"Reading dataset from file: {p3_path_to_data}...")
p3_data_json = json.load(open(p3_path_to_data, "r"))

In [ ]:
# Combine them into a single big train!
p1_examples = p1_data_json["examples"]["train"]
p2_examples = p2_data_json["examples"]["train"]
p3_data_json["examples"]["train"].extend(p1_examples)
p3_data_json["examples"]["train"].extend(p2_examples)
data_json = p3_data_json

In [ ]:
# let us downsample it to ?K
len(p3_data_json["examples"]["train"])

In [ ]:
# For generating the splits, we actually have to go through compositional splits first
# and then consider random splits like dev and test. Because, we don't want things mixed up
# in the dev and test. Dev and test should only contain commands that appear in the train,
# so a total random partition at the end should work.

# We do the splits step-by-step!
id_example_map = OrderedDict({})
id_splits_map = OrderedDict({})
index = 0
for example in data_json["examples"]["train"]:
    id_example_map[index] = example
    id_splits_map[index] = set([]) # set of splits that this example belongs to.

    # a1_novel_color_attribute
    if "yellow,square" in example['command']:
        id_splits_map[index].add("a1_novel_color_attribute")
    
    # a2_novel_color_attribute_visual
    if example["derivation"] == "$OBJ_0":
        if "red,square" in example['command'] or \
            (example['situation']['placed_objects']['0']['object']['shape'] == "square" and \
             example['situation']['placed_objects']['0']['object']['color'] == "red"):
            id_splits_map[index].add("a2_novel_color_attribute_visual")
    elif example["derivation"] == "$OBJ_0 ^ $OBJ_1":
        if "red,square" in example['command'] or \
            (example['situation']['placed_objects']['0']['object']['shape'] == "square" and \
             example['situation']['placed_objects']['0']['object']['color'] == "red") or \
            (example['situation']['placed_objects']['1']['object']['shape'] == "square" and \
             example['situation']['placed_objects']['1']['object']['color'] == "red"):
            id_splits_map[index].add("a2_novel_color_attribute_visual")
    elif example["derivation"] == "$OBJ_0 ^ $OBJ_1 & $OBJ_2":
        if "red,square" in example['command'] or \
            (example['situation']['placed_objects']['0']['object']['shape'] == "square" and \
             example['situation']['placed_objects']['0']['object']['color'] == "red") or \
            (example['situation']['placed_objects']['1']['object']['shape'] == "square" and \
             example['situation']['placed_objects']['1']['object']['color'] == "red") or \
            (example['situation']['placed_objects']['2']['object']['shape'] == "square" and \
             example['situation']['placed_objects']['2']['object']['color'] == "red"):
            id_splits_map[index].add("a2_novel_color_attribute_visual")
    else:
        pass
    
    # gscan_small_cylinder_command_target_only
    if "small,cylinder" in example['command'] or \
        "small,red,cylinder" in example['command'] or \
        "small,blue,cylinder" in example['command'] or \
        "small,yellow,cylinder" in example['command'] or \
        "small,green,cylinder" in example['command']:
        id_splits_map[index].add("a3_novel_size_attribute")
    
    # novel_yellow_square_blue_circle_coexist_shape
    if "yellow,square" in example['command'] and "blue,circle" in example['command']:
        id_splits_map[index].add("b_novel_object_coexist")

    # novel_same_shape_is_inside_coexist_relation
    if "same,shape" in example['command'] and "is,inside" in example['command']:
        id_splits_map[index].add("c_novel_relation_coexist")
        
    # novel_inside_of_as_yellow_box
    if "is,inside,of,a,yellow,box" in example['command'] or \
        "is,inside,of,the,yellow,box" in example['command'] or \
        "is,inside,of,a,small,yellow,box" in example['command'] or \
        "is,inside,of,the,small,yellow,box" in example['command'] or \
        "is,inside,of,a,big,yellow,box" in example['command'] or \
        "is,inside,of,the,big,yellow,box" in example['command']:
        id_splits_map[index].add("d_novel_object_relation_pair")
    
    if example['grammer_pattern'] == "$OBJ_0 ^ $OBJ_1 & $OBJ_2 & $OBJ_3":
        id_splits_map[index].add("e_novel_clause_length")
    
    index += 1

In [ ]:
splits_distribution = OrderedDict({})
splits_assignment = OrderedDict({})
count = 0
ccount = 0
for index, splits in id_splits_map.items():
    if len(splits) == 0:
        count += 1
        split = "train" # let us split this up later!
        if split in splits_distribution.keys():
            splits_distribution[split] += 1
        else:
            splits_distribution[split] = 1
        
        if split in splits_assignment:
            splits_assignment[split].append(index)
        else:
            splits_assignment[split] = [index]
    else:
        ccount += 1
        for split in splits:
            if split in splits_distribution.keys():
                splits_distribution[split] += 1
            else:
                splits_distribution[split] = 1
                
            if split in splits_assignment:
                splits_assignment[split].append(index)
            else:
                splits_assignment[split] = [index]

# Let us further segment train into dev and test!
all_example_id = splits_assignment["train"]
gscan_dev_size = int(len(all_example_id)*0.01)
gscan_test_size = int(len(all_example_id)*0.052)
random.shuffle(all_example_id)
train_example_id = all_example_id[:(-gscan_dev_size-gscan_test_size)]
dev_example_id = all_example_id[(-gscan_dev_size-gscan_test_size):-gscan_dev_size]
test_example_id = all_example_id[-gscan_dev_size:]
splits_assignment["train"] = train_example_id
splits_assignment["dev"] = dev_example_id
splits_assignment["test"] = test_example_id

In [ ]:
splits_distribution

In [ ]:
for split, all_ids in splits_assignment.items():
    print(f"for {split} split, we have {len(all_ids)} examples.")

In [ ]:
# remake our data file accordingly.
updated_examples = OrderedDict({})
for split, all_ids in splits_assignment.items():
    updated_examples[split] = []
    for _id in all_ids:
        updated_examples[split].append(id_example_map[_id])
# save it to the disk
data_json["examples"] = updated_examples

In [ ]:
with open("../../data-files-updated/ReaSCAN-compositional/data-compositional-splits-all.txt", "w") as fd:
    json.dump(data_json, fd, indent=4)

In [ ]:
# remake our data file accordingly.
updated_examples = OrderedDict({})
for split, all_ids in splits_assignment.items():
    if split == "train" or split == "dev" or split == "test":
        updated_examples[split] = []
        for _id in all_ids:
            updated_examples[split].append(id_example_map[_id])

In [ ]:
# save it to the disk
data_json["examples"] = updated_examples
with open("../../data-files-updated/ReaSCAN-compositional/data-compositional-splits-train.txt", "w") as fd:
    json.dump(data_json, fd, indent=4)

#### P1+P2+P3: Compositional Splits Continue
We need to make sure novel attribute splits actually require the attribute to reason, otherwise, it becomes less meaningful, and may cause accuracy inflation afterwards.

In [ ]:
# Combine all of three together
# We downsample it to make it trainable within reasonable time frame!
path_to_data = "../../data-files/ReaSCAN-compositional/data-compositional-splits-all.txt"
logger.info(f"Reading dataset from file: {path_to_data}...")
data_json = json.load(open(path_to_data, "r"))

In [ ]:
# a1 
a1_attribute_example_filtered = []
attribute_change = 0
for example in data_json["examples"]['a1_novel_color_attribute']:
    if example['has_attribute_distractor']:
        for k, v in example['object_expression'].items():
            if "yellow square" in v:
                if example['attribute_distractor_metadata'][0]['distractor_metadata'][0]['modified_obj'] == k:
                    if example['attribute_distractor_metadata'][0]['distractor_metadata'][0]['modified_attribute'] == "$COLOR":
                        a1_attribute_example_filtered += [example]
                        attribute_change += 1
    

In [ ]:
print(f"Actual examples for a1 = {attribute_change}")

In [ ]:
# a2
a2_attribute_example_filtered = []
attribute_change = 0
for example in data_json["examples"]['a2_novel_color_attribute_visual']:
    if "red,square" in example["command"]:
        if example['has_attribute_distractor']:
            for k, v in example['object_expression'].items():
                if "red square" in v:
                    if example['attribute_distractor_metadata'][0]['distractor_metadata'][0]['modified_obj'] == k:
                        if example['attribute_distractor_metadata'][0]['distractor_metadata'][0]['modified_attribute'] == "$COLOR":
                            a2_attribute_example_filtered += [example]
                            attribute_change += 1
    else:
        # this is for the visual part, we automatically added in.
        a2_attribute_example_filtered += [example]
        attribute_change += 1

In [ ]:
print(f"Actual examples for a2 = {attribute_change}")

In [ ]:
# a3
a3_attribute_example_filtered = []
attribute_change = 0
for example in data_json["examples"]['a3_novel_size_attribute']:
    if example['has_attribute_distractor']:
        for k, v in example['object_expression'].items():
            if "small" in v and "cylinder" in v:
                if example['attribute_distractor_metadata'][0]['distractor_metadata'][0]['modified_obj'] == k:
                    if example['attribute_distractor_metadata'][0]['distractor_metadata'][0]['modified_attribute'] == "$SIZE":
                        a3_attribute_example_filtered += [example]
                        attribute_change += 1

In [ ]:
print(f"Actual examples for a3 = {attribute_change}")

In [ ]:
b1_attribute_example_filtered = []
for example in data_json["examples"]['b_novel_object_coexist']:
    b1_attribute_example_filtered += [example]

In [ ]:
len(b1_attribute_example_filtered)

In [ ]:
b2_attribute_example_filtered = []
for example in data_json["examples"]['c_novel_relation_coexist']:
    b2_attribute_example_filtered += [example]

In [ ]:
len(b2_attribute_example_filtered)

In [ ]:
b3_attribute_example_filtered = []
for example in data_json["examples"]['d_novel_object_relation_pair']:
    b3_attribute_example_filtered += [example]

In [ ]:
p1_test_example_filtered = []
p2_test_example_filtered = []
p3_test_example_filtered = []
for example in data_json["examples"]["test"]:
    if example['derivation'] == "$OBJ_0":
        p1_test_example_filtered += [example]
    elif example['derivation'] == "$OBJ_0 ^ $OBJ_1":
        p2_test_example_filtered += [example]
    elif example['derivation'] == "$OBJ_0 ^ $OBJ_1 & $OBJ_2":
        p3_test_example_filtered += [example]
print(f"p1 test example count={len(p1_test_example_filtered)}")
print(f"p2 test example count={len(p2_test_example_filtered)}")
print(f"p3 test example count={len(p3_test_example_filtered)}")

In [ ]:
# Let us write each splits into a different file, so it can be loaded much faster!
data_json["examples"] = {}
data_json["examples"]["test"] = a1_attribute_example_filtered
with open("../../data-files-updated/ReaSCAN-compositional-a1/data-compositional.txt", "w") as fd:
    json.dump(data_json, fd, indent=4)

In [ ]:
data_json["examples"] = {}
data_json["examples"]["test"] = a2_attribute_example_filtered
with open("../../data-files-updated/ReaSCAN-compositional-a2/data-compositional.txt", "w") as fd:
    json.dump(data_json, fd, indent=4)

In [ ]:
data_json["examples"] = {}
data_json["examples"]["test"] = a3_attribute_example_filtered
with open("../../data-files-updated/ReaSCAN-compositional-a3/data-compositional.txt", "w") as fd:
    json.dump(data_json, fd, indent=4)

In [ ]:
data_json["examples"] = {}
data_json["examples"]["test"] = b1_attribute_example_filtered
with open("../../data-files-updated/ReaSCAN-compositional-b1/data-compositional.txt", "w") as fd:
    json.dump(data_json, fd, indent=4)

In [ ]:
data_json["examples"] = {}
data_json["examples"]["test"] = b2_attribute_example_filtered
with open("../../data-files-updated/ReaSCAN-compositional-b2/data-compositional.txt", "w") as fd:
    json.dump(data_json, fd, indent=4)

In [ ]:
data_json["examples"] = {}
data_json["examples"]["test"] = b3_attribute_example_filtered
with open("../../data-files-updated/ReaSCAN-compositional-b3/data-compositional.txt", "w") as fd:
    json.dump(data_json, fd, indent=4)

In [ ]:
# Let us write each splits into a different file, so it can be loaded much faster!
data_json["examples"] = {}
data_json["examples"]["test"] = p1_test_example_filtered
with open("../../data-files-updated/ReaSCAN-compositional-p1-test/data-compositional.txt", "w") as fd:
    json.dump(data_json, fd, indent=4)

In [ ]:
# Let us write each splits into a different file, so it can be loaded much faster!
data_json["examples"] = {}
data_json["examples"]["test"] = p2_test_example_filtered
with open("../../data-files-updated/ReaSCAN-compositional-p2-test/data-compositional.txt", "w") as fd:
    json.dump(data_json, fd, indent=4)

In [ ]:
# Let us write each splits into a different file, so it can be loaded much faster!
data_json["examples"] = {}
data_json["examples"]["test"] = p3_test_example_filtered
with open("../../data-files-updated/ReaSCAN-compositional-p3-test/data-compositional.txt", "w") as fd:
    json.dump(data_json, fd, indent=4)

#### Novel Clause Length Split

In [ ]:
path_to_data = "../../data-files-updated/ReaSCAN-compositional-p4/data-train.txt"
logger.info(f"Reading dataset from file: {path_to_data}...")
data_json = json.load(open(path_to_data, "r"))

In [ ]:
p4_test_example_filtered = data_json["examples"]["train"]
data_json["examples"] = {}
data_json["examples"]["test"] = p4_test_example_filtered
with open("../../data-files-updated/ReaSCAN-compositional-p4-test/data-compositional.txt", "w") as fd:
    json.dump(data_json, fd, indent=4)

In [ ]:
len(data_json["examples"]["test"])

#### Explore more possibilities of harder splits!

Command constant but more complex distractor sampling stratigies!

In [21]:
class Simulator(object):
    """
    This convert generated grammers into a world/situation.
    
    Sample Situation:
    Situation(grid_size=15, agent_position=Position(row=7, column=2), agent_direction=INT_TO_DIR[0],
              target_object=PositionedObject(object=Object(size=2, color='red', shape='circle'),
                                             position=Position(row=10, column=4),
                                             vector=np.array([1, 0, 1])),
              placed_objects=[PositionedObject(object=Object(size=2, color='red', shape='circle'),
                                               position=Position(row=10, column=4),
                                               vector=np.array([1, 0, 1])),
                              PositionedObject(object=Object(size=4, color='green', shape='circle'),
                                               position=Position(row=3, column=12),
                                               vector=np.array([0, 1, 0]))], carrying=None)
                                               
    Sample Placement in the World:
    world.place_object(Object(size=2, color="green", shape="box"), position=Position(row=2, column=2))
    
    """
    def __init__(self, object_vocabulary, vocabulary, grid_size=6, 
                 n_object_min=6,
                 n_object_max=12,
                 save_directory="./tmp/"):
        self.object_vocabulary = object_vocabulary
        self.vocabulary = vocabulary
        self.grid_size = grid_size
        self.n_object_min = n_object_min
        self.n_object_max = n_object_max

        self._world = World(grid_size=grid_size, colors=vocabulary.get_semantic_colors(),
                            object_vocabulary=object_vocabulary,
                            shapes=vocabulary.get_semantic_shapes(),
                            save_directory=save_directory)
        self._world.clear_situation()
    
    def sample_object_shape(
        self, obj_grammer, obj_pattern, obj_str, rel_map, 
        is_root, shape_map
    ):
        obj_pattern = obj_pattern.split(" ")
        obj_str = obj_str.split(" ")
        shape = None
        if len(obj_str) == 3:
            shape = obj_str[2]
        elif len(obj_str) == 2:
            shape = obj_str[1]
        elif len(obj_str) == 1:
            # it must be the object
            shape = obj_str[0]
        # Final handling for the shape.
        if shape == "object":
            shape = self.object_vocabulary.sample_shape()
            
        # Override size, color and shape based on relations.
        if not is_root:
            # Go through the rel.
            for pair, rel in rel_map.items():
                if obj_grammer == pair[-1]:
                    if pair[0] in shape_map.keys():
                        # if this obj is acting as a child node
                        # then have to complain with parent node
                        if rel == "$SAME_SHAPE":
                            shape = shape_map[pair[0]]
                        elif rel == "$IS_INSIDE":
                            shape = "box"
        return shape
    
    def sample_object_spec(
        self, obj_grammer, obj_pattern, obj_str, rel_map, 
        is_root, obj_placed_map, 
        size_restriction_map=None,
        mentioned_shapes=None
    ):
        obj_pattern = obj_pattern.split(" ")
        obj_str = obj_str.split(" ")
        color = None
        size = None
        shape = None
        if len(obj_str) == 3:
            size = self.object_vocabulary.sample_size_with_prior(prior=obj_str[0])
            color = obj_str[1]
            shape = obj_str[2]
        elif len(obj_str) == 2:
            if "$COLOR" in obj_pattern: # color + shape.
                size = self.object_vocabulary.sample_size()
                color = obj_str[0]
                shape = obj_str[1]
            elif "$SIZE" in obj_pattern: # size + shape.
                size = self.object_vocabulary.sample_size_with_prior(prior=obj_str[0])
                color = self.object_vocabulary.sample_color()
                shape = obj_str[1]
        elif len(obj_str) == 1:
            # it must be the object
            size = self.object_vocabulary.sample_size()
            color = self.object_vocabulary.sample_color()
            shape = obj_str[0]
        # Final handling for the shape.
        if shape == "object":
            # WARNING: this is a corner case you will hit
            # if your logic chain is long, you may need to
            # consider remove object option!
            if mentioned_shapes != None and len(mentioned_shapes) == self.object_vocabulary._shapes:
                assert False
            if is_root:
                shape = self.object_vocabulary.sample_shape() # _exclude=mentioned_shapes
            else:
                shape = self.object_vocabulary.sample_shape()
                
        return Object(color=color,size=size,shape=shape)
                    
    def sample_object_position(
        self, sampled_obj, root, obj_grammer, 
        rel_map, obj_placed_map, 
        obj_position_map
    ):
        """
        Return:
        -1: No position can be sampled.
        """
        # If it is the first node, we directly return.
        if sampled_obj.shape != "box":
            obj_random_pos = self._world.sample_position_complex(
                condition="normal", sample_one=True
            )
        else:
            obj_random_pos = self._world.sample_position_complex(
                condition="box", box_size=sampled_obj.size, sample_one=True
            )
        if obj_grammer == root:
            return obj_random_pos # for this round, the root node can be placed anywhere!
        
        # For some relations, we might need to resample positions!
        for pair, rel in rel_map.items():
            if obj_grammer == pair[-1]:
                if not pair[0] in obj_placed_map.keys():
                    assert False # this should never be the case! the position sampling in from top to bottom!

                # if this obj is acting as a child node
                # then have to complain with parent node
                if rel == "$SAME_ROW":
                    row = obj_position_map[pair[0]].row
                    for i in range(0, self.grid_size):
                        proposed_position = Position(row=row, column=i)
                        if not self._world.position_taken(proposed_position, condition="normal"):
                            return proposed_position
                    return -1 # too many objs
                if rel == "$SAME_COLUMN":
                    col = obj_position_map[pair[0]].column
                    for i in range(0, self.grid_size):
                        proposed_position = Position(row=i, column=col)
                        if not self._world.position_taken(proposed_position, condition="normal"):
                            return proposed_position
                    return -1 # too many objs
                elif rel == "$IS_INSIDE":
                    # we need to make sure enclosure
                    assert sampled_obj.shape == "box"
                    size = sampled_obj.size
                    potential_positions = []
                    row = obj_position_map[pair[0]].row
                    col = obj_position_map[pair[0]].column
                    for i in range(0, self.grid_size-size+1):
                        for j in range(0, self.grid_size-size+1):
                            # we need to cover the is inside obj
                            if row >= i and row < i + sampled_obj.size and \
                                col >= j and col < j + sampled_obj.size:
                                proposed_position = Position(row=i, column=j)
                                if not self._world.position_taken(proposed_position, condition="box"):
                                    potential_positions.append(Position(row=i, column=j))
                    random.shuffle(potential_positions)
                    if len(potential_positions) < 1:
                        return -1
                    return potential_positions[0]

        return obj_random_pos
    
    def sample_random_object_spec(
        self, 
        size_exclude=None, 
        color_exclude=None, shape_exclude=None, combo_exclude=None
    ):
        if combo_exclude:
            while True:
                d_size = self.object_vocabulary.sample_size(_exclude=size_exclude)
                d_color = self.object_vocabulary.sample_color(_exclude=color_exclude)
                if shape_exclude != None and len(shape_exclude) >= 3:
                    shape_exclude = None # let us not excluding anything, and let graph matching reject this!
                d_shape = self.object_vocabulary.sample_shape(_exclude=shape_exclude)
                if d_color + " " + d_shape not in combo_exclude:
                    return Object(color=d_color,size=d_size,shape=d_shape)
        else:
            d_size = self.object_vocabulary.sample_size(_exclude=size_exclude)
            d_color = self.object_vocabulary.sample_color(_exclude=color_exclude)
            if shape_exclude != None and len(shape_exclude) >= 3:
                shape_exclude = None # let us not excluding anything, and let graph matching reject this!
            d_shape = self.object_vocabulary.sample_shape(_exclude=shape_exclude)
            return Object(color=d_color,size=d_size,shape=d_shape)
    
    def place_distractor_from_dict(
        self, distractors_dict, 
        obj_placed_map, obj_position_map, 
        debug=False, 
        special_shape_size_bound=None,
        mentioned_shapes=None,
    ):
        if debug:
            import pprint
            pp = pprint.PrettyPrinter(indent=4)
            pp.pprint(distractors_dict)
        distractor_root = f"$OBJ_{len(obj_placed_map)}"
        success = True
        distractors_obj_map = distractors_dict["obj_map"]
        distractors_rel_map = distractors_dict["rel_map"]
        distractors_obj_pattern_map = distractors_dict["obj_pattern_map"]
        distractors_size_map = distractors_dict["size_map"]
        
        distractors_sampled_obj_map = {}
        for dis_grammer, dis_str in distractors_obj_map.items():
            # 1. Sample object.
            sampled_dis = self.sample_object_spec(
                dis_grammer,
                distractors_obj_pattern_map[dis_grammer], 
                dis_str, distractors_rel_map, 
                is_root=dis_grammer==distractor_root, 
                obj_placed_map=obj_placed_map,
                mentioned_shapes=mentioned_shapes,
            )
            # 1.1. Update the size of the object if needed.
            if dis_grammer in distractors_size_map.keys():
                sampled_dis = Object(
                    color=sampled_dis.color,
                    size=distractors_size_map[dis_grammer],
                    shape=sampled_dis.shape
                )
            # 1.2. Another pass of override by using global constraints.
            special_shape_super = sampled_dis.shape
            special_shape_sub = sampled_dis.color + " " + sampled_dis.shape
            # e.g., small circle exists in the command, then any colored circle needs to be constrain
            if special_shape_super in special_shape_size_bound.keys():
                if "small" in dis_str:
                    updated_size = special_shape_size_bound[special_shape_super][0]
                else:
                    updated_size = special_shape_size_bound[special_shape_super][1]
                sampled_dis = Object(
                    color=sampled_dis.color,
                    size=updated_size,
                    shape=sampled_dis.shape
                )
            elif special_shape_sub in special_shape_size_bound.keys():
                if "small" in dis_str:
                    updated_size = special_shape_size_bound[special_shape_sub][0]
                else:
                    updated_size = special_shape_size_bound[special_shape_sub][1]
                sampled_dis = Object(
                    color=sampled_dis.color,
                    size=updated_size,
                    shape=sampled_dis.shape
                )
            else:
                pass # Do nothing.
            distractors_sampled_obj_map[dis_grammer] = sampled_dis
        
        # 2. Update it using relationships.
        for pair, rel in distractors_rel_map.items():
            if rel == "$SAME_SHAPE":
                pass
            elif rel == "$SAME_COLOR":
                pass
            elif rel == "$SAME_SIZE":
                # Update the src node size information.
                size = distractors_sampled_obj_map[pair[1]].size
                distractors_sampled_obj_map[pair[0]] = Object(
                    color=distractors_sampled_obj_map[pair[0]].color,
                    size=size,
                    shape=distractors_sampled_obj_map[pair[0]].shape
                )
            elif rel == "$IS_INSIDE":
                pass # Do nothing!

        placed_dis_grammer = []
        for dis_grammer, sampled_dis in distractors_sampled_obj_map.items():
            # 2. Place on the world map.
            sampled_pos = self.sample_object_position(
                sampled_dis, distractor_root, 
                dis_grammer, distractors_rel_map, 
                obj_placed_map, obj_position_map
            )

            if sampled_dis == -1 or sampled_pos == -1:
                # We allow partial placement as they add difficulties!
                return False

            return_code = self._world.place_object(
                sampled_dis, 
                position=sampled_pos, target=False # Distractor is never the target!
            )
            if return_code == -1:
                assert False # Due to our design, this should never happen!
            obj_placed_map[dis_grammer] = sampled_dis
            obj_position_map[dis_grammer] = sampled_pos
            
        return True
    
    def sample_situations_from_grounded_grammer(
        self, grammer_pattern, 
        obj_pattern_map, rel_map, obj_map, root="$OBJ_0", 
        is_plot=False, 
        include_random_distractor=False, 
        include_relation_distractor=False, 
        include_attribute_distractor=False, 
        include_isomorphism_distractor=False, 
        full_relation_probability=0.5,
        debug=False,
    ):
        # Clear current world.
        self._world.clear_situation()
        
        # Start placing objects with specs.
        obj_placed_map = OrderedDict({})
        obj_position_map = OrderedDict({})
        referred_obj = root
        
        # Preliminary size check!
        """
        Here is a list of potential internal conflicts:
        (1) ... to a small box ... to a yellow box ...
        Explain: we need to adjust the size of two boxes
        so that small box has 1 size, and all other boxes 
        have the same other size.
        There will at max two different size of same type objects.
        
        So this is the rule:
        For 1 type of shape, max two different sizes.
        """
        # Ok, we need to determine shapes first!
        # Even there is any abstract object, the
        # shape is now determined.
        object_map = {}
        mentioned_shapes = set([]) # this is used to sample shapes for object.
        for obj_grammer, obj_str in obj_map.items():
            shape = self.extract_shape(obj_str)
            if shape != "":
                mentioned_shapes.add(shape) # For abstract mention, shape is determined later!
        for obj_grammer, obj_str in obj_map.items():
            # 1. Sample object.
            sampled_obj = self.sample_object_spec(
                obj_grammer,
                obj_pattern_map[obj_grammer], obj_str, rel_map, 
                is_root=obj_grammer==root, 
                obj_placed_map=object_map,
                mentioned_shapes=mentioned_shapes,
            )
            object_map[obj_grammer] = sampled_obj
        
        # Next, we update all of them based on relations.
        # Final pass, we need to change attributes of objects based
        # on relations.
        # Here, we only change size!
        for pair, rel in rel_map.items():
            if rel == "$SAME_SHAPE":
                # Update the src node shape information.
                shape = object_map[pair[1]].shape
                object_map[pair[0]] = Object(
                    color=object_map[pair[0]].color,
                    size=object_map[pair[0]].size,
                    shape=shape
                )
                pass
            elif rel == "$SAME_COLOR":
                # Update the src node color information.
                color = object_map[pair[1]].color
                object_map[pair[0]] = Object(
                    color=color,
                    size=object_map[pair[0]].size,
                    shape=object_map[pair[0]].shape
                )
                pass
            elif rel == "$SAME_SIZE":
                # Update the src node size information.
                size = object_map[pair[1]].size
                object_map[pair[0]] = Object(
                    color=object_map[pair[0]].color,
                    size=size,
                    shape=object_map[pair[0]].shape
                )
            elif rel == "$IS_INSIDE":
                pass
            
        # Then, we will determine size bounds.
        special_shape_size_bound = {}
        for obj_grammer, obj_pattern in obj_pattern_map.items():
            
            small_size = random.randint(
                self.object_vocabulary._min_size, 
                self.object_vocabulary._max_size-1
            )
            big_size = random.randint(
                small_size+1, 
                self.object_vocabulary._max_size
            )
            
            if "$SIZE" in obj_pattern and "$COLOR" in obj_pattern:
                special_shape = object_map[obj_grammer].color + " " + object_map[obj_grammer].shape
                if object_map[obj_grammer].shape in special_shape_size_bound.keys():
                    # e.g., small circle exists
                    special_shape_size_bound[special_shape] = special_shape_size_bound[object_map[obj_grammer].shape]
                else:
                    # e.g., small yellow circle
                    special_shape_size_bound[special_shape] = [small_size, big_size]
            elif "$SIZE" in obj_pattern and not "$COLOR" in obj_pattern:
                # e.g., small circle
                # overwrite any existing bounds.
                special_shape = object_map[obj_grammer].shape
                for ss, bound in special_shape_size_bound.items():
                    if special_shape in ss:
                        special_shape_size_bound[ss] = [small_size, big_size]
                # for shape, it adds.
                special_shape_size_bound[special_shape] = [small_size, big_size]
                # for non-sized shape, it also adds as long as shape is the same.
                for obj_grammer, obj_pattern in obj_pattern_map.items():
                    if special_shape in obj_map[obj_grammer]:
                        if "$COLOR" in obj_pattern:
                            special_shape = object_map[obj_grammer].color + " " + object_map[obj_grammer].shape
                            special_shape_size_bound[special_shape] = [small_size, big_size]
            else:
                continue
        
        # Update object size based on global scanning results.
        updated_object_map = {}
        for obj_grammer, obj_pattern in obj_pattern_map.items():

            special_shape_super = object_map[obj_grammer].shape
            special_shape_sub = object_map[obj_grammer].color + " " + object_map[obj_grammer].shape
            
            # e.g., small circle exists in the command, then any colored circle needs to be constrain
            if special_shape_super in special_shape_size_bound.keys():
                if "small" in obj_map[obj_grammer]:
                    updated_size = special_shape_size_bound[special_shape_super][0]
                else:
                    updated_size = special_shape_size_bound[special_shape_super][1]
                updated_object_map[obj_grammer] = Object(
                    color=object_map[obj_grammer].color,
                    size=updated_size,
                    shape=object_map[obj_grammer].shape
                )
            elif special_shape_sub in special_shape_size_bound.keys():
                if "small" in obj_map[obj_grammer]:
                    updated_size = special_shape_size_bound[special_shape_sub][0]
                else:
                    updated_size = special_shape_size_bound[special_shape_sub][1]
                updated_object_map[obj_grammer] = Object(
                    color=object_map[obj_grammer].color,
                    size=updated_size,
                    shape=object_map[obj_grammer].shape
                )
            else:
                # If nothing exists in the special size map, then we don't need
                # to alter the size.
                updated_object_map[obj_grammer] = object_map[obj_grammer]

        # Final pass, we need to change attributes of objects based
        # on relations.
        # Here, we only change size!
        for pair, rel in rel_map.items():
            if rel == "$SAME_SHAPE":
                pass
            elif rel == "$SAME_COLOR":
                pass
            elif rel == "$SAME_SIZE":
                # Update the src node size information.
                size = updated_object_map[pair[1]].size
                updated_object_map[pair[0]] = Object(
                    color=object_map[pair[0]].color,
                    size=size,
                    shape=object_map[pair[0]].shape
                )
            elif rel == "$IS_INSIDE":
                pass
        
        # Next, we sample positions of all objects and place them.
        for obj_grammer, obj_str in obj_map.items():
            # 1. Sample object (bu fetching the updated one).
            sampled_obj = updated_object_map[obj_grammer]
            
            # 2. Place on the world map.
            sampled_pos = self.sample_object_position(
                sampled_obj, root, obj_grammer, rel_map, 
                obj_placed_map, obj_position_map
            )
            
            if sampled_obj == -1 or sampled_pos == -1:
                assert False # we can assert false as it is impossible!
        
            self._world.place_object(
                sampled_obj, 
                position=sampled_pos, target=obj_grammer==root
            )
            obj_placed_map[obj_grammer] = sampled_obj
            obj_position_map[obj_grammer] = sampled_pos
        
        # Update the shape mentions for later use.
        for obj_grammer, obj in obj_placed_map.items():
            mentioned_shapes.add(obj.shape)
            # maybe let us add color + shape to be more specific
            mentioned_shapes.add(obj.color + " " + obj.shape)
            # we also need to be more strict.
            # if we have something like blue object
            # $COLOR + $ABS_SHAPE, then
            # we cannot have that color for any shape i think!
            if "$COLOR" in obj_pattern_map[obj_grammer] and \
                "$ABS_SHAPE" in obj_pattern_map[obj_grammer]:
                for shape in self.object_vocabulary._shapes:
                    mentioned_shapes.add(obj.color + " " + shape)
            
        
        """
        Distractor Sampling Strategies and Design
        
        Giving a complex command as:
        "the small red circle(1) that is in the same row(a) 
        as a big green square(2) and that is in the same column(b) 
        as a small yellow cylinder(3)."
        
        We have 4 types of distractors (objects):
        - Attribute-based Distractors
        - Relation-based Distractors
        - Sytax-based Distractors
        - Random Distractors
        
        For each type of distractors, we will modify the command
        to generate a new command for distractors. Then, we will
        ensure such every command-world pair needs to reason about
        attribute, relation and syntax. 
        
        There are some caveats around this design. Due to the 
        complexity of the command, to make sure
        every attribute/relation is necessary becomes unfeasible. For
        example, if we want to make "small" in "the small red circle (1)"
        necessary, then, we need to put another non-"small" "red circle".
        This is easy. However, if we want to make "big" in "the big
        green square" necessary, we essentially need to sample another
        set of objects (at max 3) that complies with a modified command
        "the small red circle(1) that is in the same row(a) 
        as a small green square(2*) and that is in the same column(b) 
        as a small yellow cylinder(3)."
        
        Following this logic, if we want to make sure *every descriptor*\
        (i.e., every adjective) is necessary to identity the referent
        target, we could flood the system easily with way too many
        distractors that cannot fit in our grid world.
        
        On the other hand, the goal of having the distractors is to
        have the system learn the importantce of relations, attributes
        and linguistic syntax. So, are these distractors necessary?
        Do we need to actually have an exhaustive list of distractors
        for each command-world pair in order to have the model to learn
        this? We propose the answer is No, but Yes in the dataset level. In
        the command level, we will not make sure *every descriptor* is
        necessary, but in the command level, we will make sure 
        *every descriptor* matters for at least some of the command.
        Otherwise, the model may just completely ignores one part of
        the command and relies on the rest.
        
        In our design, we ensure for each command-world pair, some attribute
        and some relation and some syntax are needed. In the dataset
        level, we ensure different attribute, relation and syntax are 
        weighted equally.
        
        We propose to sample distractors following the design below:
        
        For a command such as
        "A that is X B and that is Y C"
        (1) We generate two distractor commands: "A that X B"; and "A that Y C"
        without guarantee all relations in the original command. This samples
        4 distractors. This ensures X and Y are necessary!
        
        (2) Next, we need to ensure that if we change some descriptors for
        A, B or C, referent target cannot be identified. For example, if
        we change B from "yellow square" to "blue square" the referent target
        should change. In this case, we need to sample a new set of {A,B,C}.
        And if we do this for each object, this results in 9 new distractors.
        If size is not selected, we potentially need 3 more distractors to
        ground the size aspects.
        
        (3) Next, to ensure model learns linguistic syntax, instead of simple
        BoW approach to represent the command, we would perform swap attributes
        between objects. We pick a pair of objects, and swap attributes 
        randomly. This results in 3 more distractors.
        
        (1) + (2) + (3) results in at max 19 distractors for each command-world pair.
        Plus the original 3 objects, we have in total 21 distractors.
        This is still a lot higher than gSCAN which is at max about 12.
        
        Then, we design another way to sample distractors:
        (1) We pick 1 relations from {X, Y}, and generate distracotrs: 3 distractors.
        
        (2) We pick 1 object from {A, B, C} and modify its attribute, sample 3 distractors.
        if size is not selected for any object, we need to randomly sample non-relational
        counterparts, at max 3.
        
        (3) Same, so 3.
        
        (1) + (2) + (3) results in 3 + 3 + 3 + 3 = 12, 12 + 3 -> at max 15. Is this doable?
        
        Test set. global v.s. local compositional generalization. In the test set, we 
        can pick different/more aspect of differeent/more obj that matter for the
        correctly reasonings, and generate test cases with  more distractors.
        """
        
        """
        Calling in this way to create distractors:

        simulator.sample_distractor_grammer_by_relation(
            grammer_pattern, 
            obj_pattern_map, 
            rel_map, 
            obj_map, 
            sampled_world
        )
        """
        temp_sampled_world = {
            "obj_map" : obj_placed_map,
            "pos_map" : obj_position_map,
            "referred_obj" : referred_obj,
            "situation" : copy.deepcopy(self._world.get_current_situation())
        }
        
        # Three types of distractor sampling for different purposes:
        # sample_distractor_grammer_by_relation()
        # - We will edit one leaf node, so that it makes sure
        #   the command is necessary!
        # sample_distractor_grammer_by_size()
        # - Size relatives need to be meaningful. We will add relational
        #   objects to make sure.
        # sample_distractor_grammer_by_isomorphism()
        # - This is to ensure syntax learning.
        
        distractor_switch_map = OrderedDict({
            "relation" : [],
            "attribute" : False,
            "isomorphism" : False, 
            "random" : False,
        })
        relation_distractors_dicts = [{
            "distractor_metadata": {}
        }]
        attribute_distractors_dicts = [{
            "distractor_metadata": {}
        }]
        isomorphism_distractors_dicts = [{
            "distractor_metadata": {}
        }]
        
        if random.random() < full_relation_probability:
            full_relation_set=True
        else:
            full_relation_set=False
            
        obj_drafted_count = len(obj_placed_map)
        if include_relation_distractor:
            """
            Relation Distractors: Count=3*n, at max 6.
            Relation Distractors (fast): Count=2*n, at max 4.
            
            sample_distractor_grammer_by_relation_fast
            
            sample_distractor_grammer_by_relation
            """
            relation_distractors_dicts = self.sample_distractor_grammer_by_relation_fast(
                grammer_pattern, 
                obj_pattern_map, 
                rel_map, 
                obj_map, 
                temp_sampled_world,
                obj_base_count=len(obj_placed_map),
                full_set=full_relation_set,
            )
            if len(relation_distractors_dicts) == 0:
                pass # Size distractor is not applicable 
            else:
                distractor_switch = []
                is_full_relation = True
                for distractors_dict in relation_distractors_dicts:
                    obj_drafted_count += len(distractors_dict["obj_map"])
                    succeed = self.place_distractor_from_dict(
                        distractors_dict, 
                        obj_placed_map, 
                        obj_position_map,
                        debug=debug,
                        special_shape_size_bound=special_shape_size_bound,
                        mentioned_shapes=mentioned_shapes,
                        # This is needed as maybe distractors also 
                        # need to be bounded by global constraints.
                    )
                    if succeed:
                        distractor_switch_map["relation"].append(True)
                    else:
                        distractor_switch_map["relation"].append(False)

        if include_attribute_distractor:
            """
            Attribution Distractors: Count=3-6.
            """
            # If the command is small, we can overwrite this
            full_set = not full_relation_set
            attribute_distractors_dicts = self.sample_distractor_grammer_by_attribute(
                grammer_pattern, 
                obj_pattern_map, 
                rel_map, 
                obj_map, 
                temp_sampled_world,
                special_shape_size_bound,
                obj_base_count=obj_drafted_count, # This is important, as previous draft may success but placement can fail!
                full_set=True, # always include some attribute based distractors.
            )
            if len(attribute_distractors_dicts) == 0:
                pass # Size distractor is not applicable 
            else:
                obj_drafted_count += len(attribute_distractors_dicts[0]["obj_map"])
                succeed = self.place_distractor_from_dict(
                    attribute_distractors_dicts[0], 
                    obj_placed_map, 
                    obj_position_map,
                    debug=debug,
                    special_shape_size_bound=special_shape_size_bound,
                    mentioned_shapes=mentioned_shapes,
                    # This is needed as maybe distractors also 
                    # need to be bounded by global constraints.
                )
                if succeed:
                    if attribute_distractors_dicts[0]["grammer_pattern"] != "DUMMY":
                        distractor_switch_map["attribute"] = True # If one time it is true, it is true.
                else:
                    pass
        else:
            # we need to ground size.
            attribute_distractors_dicts = self.sample_distractor_grammer_by_attribute(
                grammer_pattern, 
                obj_pattern_map, 
                rel_map, 
                obj_map, 
                temp_sampled_world,
                special_shape_size_bound,
                obj_base_count=obj_drafted_count, # This is important, as previous draft may success but placement can fail!
                full_set=False,
            )
            if len(attribute_distractors_dicts) == 0:
                pass # Size distractor is not applicable 
            else:
                obj_drafted_count += len(attribute_distractors_dicts[0]["obj_map"])
                succeed = self.place_distractor_from_dict(
                    attribute_distractors_dicts[0], 
                    obj_placed_map, 
                    obj_position_map,
                    debug=debug,
                    special_shape_size_bound=special_shape_size_bound,
                    mentioned_shapes=mentioned_shapes,
                    # This is needed as maybe distractors also 
                    # need to be bounded by global constraints.
                )
                if succeed:
                    if attribute_distractors_dicts[0]["grammer_pattern"] != "DUMMY":
                        distractor_switch_map["attribute"] = True # If one time it is true, it is true.
                else:
                    pass
        
        if include_isomorphism_distractor:
            """
            Syntax Distractors: Count=3.
            """
            isomorphism_distractors_dicts = self.sample_distractor_grammer_by_isomorphism(
                grammer_pattern, 
                obj_pattern_map, 
                rel_map, 
                obj_map, 
                temp_sampled_world,
                obj_base_count=obj_drafted_count # This is important, as previous draft may success but placement can fail!
            )
            if len(isomorphism_distractors_dicts) == 0:
                pass # Size distractor is not applicable 
            else:
                obj_drafted_count += len(isomorphism_distractors_dicts[0]["obj_map"])
                succeed = self.place_distractor_from_dict(
                    isomorphism_distractors_dicts[0], 
                    obj_placed_map, 
                    obj_position_map,
                    debug=debug,
                    special_shape_size_bound=special_shape_size_bound,
                    mentioned_shapes=mentioned_shapes,
                    # This is needed as maybe distractors also 
                    # need to be bounded by global constraints.
                )
                if succeed:
                    distractor_switch_map["isomorphism"] = True
        
        # Probably never need this!
        """
        Random Distractors.
        """
        # Place random distractors. These are gSCAN like distractors
        # which are often not very meaningful for testing agents language
        # knowledge. We recommand always turn this off and use other
        # relation-based distractor sampling strategies.
        
        random_distractor_metadata = {}
        n_random_distractor = -1
        if include_random_distractor:
            if len(obj_placed_map) >= self.n_object_max:
                pass # Do nothing!
            else:
                n_distractor = self.n_object_max-len(obj_placed_map) # for simplier command, maybe all of them will be random?
                n_random_distractor = n_distractor
                core_obj_count = obj_drafted_count
                for i in range(0, n_distractor):
                    distractor_idx = core_obj_count+i
                    distractor_name = f"$OBJ_{distractor_idx}"

                    # Let us only sample shapes that are not exist
                    sampled_distractor = self.sample_random_object_spec(
                        shape_exclude=list(mentioned_shapes),
                        combo_exclude=list(mentioned_shapes)
                    )
                    
                    # Ok, we need to consider global size constraint!
                    special_shape_super = sampled_distractor.shape
                    special_shape_sub = sampled_distractor.color + " " +sampled_distractor.shape

                    # e.g., small circle exists in the command, then any colored circle needs to be constrain
                    size_idx = -1
                    if special_shape_super in special_shape_size_bound.keys():
                        size_idx = random.randint(0,1)
                        updated_size = special_shape_size_bound[special_shape_super][size_idx]
                        sampled_distractor = Object(
                            color=sampled_distractor.color,
                            size=updated_size,
                            shape=sampled_distractor.shape
                        )
                    elif special_shape_sub in special_shape_size_bound.keys():
                        size_idx = random.randint(0,1)
                        updated_size = special_shape_size_bound[special_shape_sub][size_idx]
                        sampled_distractor = Object(
                            color=sampled_distractor.color,
                            size=updated_size,
                            shape=sampled_distractor.shape
                        )
                    
                    if sampled_distractor.shape == "box":
                        sampled_dis_pos = self._world.sample_position_complex(
                            condition="box", box_size=sampled_distractor.size, sample_one=True
                        )
                    else:
                        sampled_dis_pos = self._world.sample_position_complex(
                            condition="normal", sample_one=True
                        )
                    
                    self._world.place_object(
                        sampled_distractor, 
                        position=sampled_dis_pos, target=False
                    )
                    obj_placed_map[distractor_name] = sampled_distractor
                    obj_position_map[distractor_name] = sampled_dis_pos
                    size_str = ""
                    if size_idx != -1:
                        size_str = "big" if size_idx == 1 else "small"
                    random_distractor_metadata[distractor_name] = " ".join([
                        size_str,
                        sampled_distractor.color,
                        sampled_distractor.shape
                    ])
                distractor_switch_map["random"] = True

        agent_position = self._world.sample_position_complex(
                            condition="normal", sample_one=True
                        )
        self._world.place_agent_at(agent_position)
        if is_plot:
            _ = self._world.render_simple()
        
        situation_snapshot = copy.deepcopy(self._world.get_current_situation())
        
        return {
            "obj_map" : obj_placed_map,
            "pos_map" : obj_position_map,
            "obj_pattern_map" : obj_pattern_map,
            "referred_obj" : referred_obj,
            "situation" : situation_snapshot, 
            "distractor_switch_map" : distractor_switch_map,
            "relation_distractor_metadata" : [
                {
                    "distractor_metadata": md["distractor_metadata"],
                    # "obj_map": md["obj_map"],
                    # "rel_map": md["rel_map"],
                } for md in relation_distractors_dicts
            ],
            "attribute_distractor_metadata" : [
                {
                    "distractor_metadata": md["distractor_metadata"],
                    # "obj_map": md["obj_map"],
                    # "rel_map": md["rel_map"],
                } for md in attribute_distractors_dicts
            ],
            "isomorphism_distractor_metadata" : [
                {
                    "distractor_metadata": md["distractor_metadata"],
                    # "obj_map": md["obj_map"],
                    # "rel_map": md["rel_map"],
                } for md in isomorphism_distractors_dicts
            ],
            "random_distractor_metadata" : [random_distractor_metadata],
            "n_random_distractor" : n_random_distractor
        }
    
    def get_action_list(
        self,
        verb=None,
        adverb=None,
    ):
        pass
    
    def extract_size(self, obj_str):
        obj_descriptors = obj_str.split(" ")
        for descriptor in obj_descriptors:
            if descriptor in ["small", "big"]:
                return descriptor
        return ""

    def extract_color(self, obj_str):
        obj_descriptors = obj_str.split(" ")
        for descriptor in obj_descriptors:
            if descriptor in self.object_vocabulary.object_colors:
                return descriptor
        return ""
    
    def extract_shape(self, obj_str):
        obj_descriptors = obj_str.split(" ")
        for descriptor in obj_descriptors:
            if descriptor in self.object_vocabulary.object_shapes:
                return descriptor
        return ""

    def convert_object_str_to_grammer(self, obj_str):
        size_g = False
        color_g = False
        abs_shape_g = False

        obj_descriptors = obj_str.split(" ")
        for descriptor in obj_descriptors:
            if descriptor in ["small", "big"]:
                size_g = True
            elif descriptor in self.object_vocabulary.object_colors:
                color_g = True
            elif descriptor in self.object_vocabulary.object_shapes:
                pass
            elif descriptor == "object":
                abs_shape_g = True

        grammer = []
        if size_g:
            grammer.append("$SIZE")
        if color_g:
            grammer.append("$COLOR")
        if abs_shape_g:
            grammer.append("$ABS_SHAPE") # Mark as deprecated!
        else:
            grammer.append("$SHAPE")
        
        return " ".join(grammer)

    def snap_pattern_to_referent_map(self, distractor_grammer_pattern, base_count):
        distractor_grammer_pattern_snapped = []
        for item in distractor_grammer_pattern.split(" "):
            if item.startswith("$"):
                new_id = int(item.split("_")[1])+base_count
                distractor_grammer_pattern_snapped.append(f"$OBJ_{new_id}")
            else:
                distractor_grammer_pattern_snapped.append(item)
        return " ".join(distractor_grammer_pattern_snapped)

    def snap_object_map_to_referent_map(self, distractor_map, base_count):
        distractor_map_snapped = OrderedDict({})
        for obj_name, item in distractor_map.items():
            new_id = int(obj_name.split("_")[1])+base_count
            new_obj_name = f"$OBJ_{new_id}"
            distractor_map_snapped[new_obj_name] = item
        return distractor_map_snapped

    def snap_relation_map_to_referent_map(self, distractor_rel_map, base_count):
        distractor_rel_map_snapped = OrderedDict({})
        for edge, item in distractor_rel_map.items():
            if edge[0].startswith("$"):
                new_id_left = int(edge[0].split("_")[1])+base_count
                new_obj_name_left = f"$OBJ_{new_id_left}"
            else:
                new_obj_name_left = edge[0]
            
            if edge[1].startswith("$"):
                new_id_right = int(edge[1].split("_")[1])+base_count
                new_obj_name_right = f"$OBJ_{new_id_right}"
            else:
                new_obj_name_right = edge[1]
            distractor_rel_map_snapped[(new_obj_name_left, new_obj_name_right)] = item
        return distractor_rel_map_snapped

    def sample_distractor_grammer_by_relation_fast(
        self, 
        referent_grammer_pattern, 
        referent_obj_pattern_map,
        referent_rel_map,
        referent_obj_map, 
        sampled_world,
        obj_base_count=0,
        full_set=True,
        stress_test=False, # Open this for our own purpose only.
    ):
        """
        You can choose between two versions.
        In this fast version, we only sample objects for the 
        selected edge.
        """
        distractors_dicts = []
        # We first collect all the relations
        relation_edges = []
        for edge, relation in referent_rel_map.items():
            # I don't want to sample box again
            # this will add too many box at the end!
            relation_edges.append(edge)
        # if there is only 1 relation, do we really need it?
        # probably not?
        if stress_test:
            if len(relation_edges) == 1:
                for i in range(10):
                    # for this, we simple sample another obj_0?
                    distractor_size_map = {}
                    distractor_obj_pattern_map = {}
                    distractor_obj_map = {}
                    distractor_rel_map = OrderedDict({})

                    distractor_size_map["$OBJ_0"] = int(sampled_world["obj_map"]["$OBJ_0"].size)
                    distractor_obj_pattern_map["$OBJ_0"] = referent_obj_pattern_map["$OBJ_0"]
                    distractor_obj_map["$OBJ_0"] = referent_obj_map["$OBJ_0"]
                    distractor_grammer_pattern = referent_grammer_pattern.split(" ")[0]

                    distractor_metadata = {
                        "edge" : "null",
                        "relation_old_type" : "null", # omit relations
                        "full_set" : True,
                    }

                    distractors_dicts += [{
                                            "grammer_pattern" : self.snap_pattern_to_referent_map(
                                                distractor_grammer_pattern,
                                                obj_base_count
                                            ),
                                            "obj_pattern_map" : self.snap_object_map_to_referent_map(
                                                distractor_obj_pattern_map,
                                                obj_base_count
                                            ),
                                            "rel_map" : self.snap_relation_map_to_referent_map(
                                                distractor_rel_map,
                                                obj_base_count
                                            ),
                                            "obj_map" : self.snap_object_map_to_referent_map(
                                                distractor_obj_map,
                                                obj_base_count
                                            ),
                                            "size_map" : self.snap_object_map_to_referent_map(
                                                distractor_size_map,
                                                obj_base_count
                                            ),
                                            "distractor_metadata" : distractor_metadata
                                        }]
                    obj_base_count += len(distractor_obj_pattern_map)
            elif len(relation_edges) == 2:
                for i in range(0, 2):
                    existing_relations = set([v for k, v in referent_rel_map.items()])
                    for selected_leaf_edge in relation_edges:

                        distractor_metadata = {
                            "edge" : selected_leaf_edge,
                            "relation_old_type" : referent_rel_map[selected_leaf_edge], # omit relations
                            "full_set" : full_set,
                        }

                        # now for the selected edge, we omit them!
                        distractor_grammer_pattern = referent_grammer_pattern.split(" ")[:-2]
                        distractor_grammer_pattern = " ".join(distractor_grammer_pattern) # omiting the last.

                        distractor_size_map = {}

                        # First, let us make copies.
                        distractor_obj_pattern_map = {}
                        keeping_edges = []
                        keeping_objects = []
                        for keeping_edge in relation_edges:
                            if keeping_edge != selected_leaf_edge:
                                keeping_edges.append(keeping_edge)
                                if keeping_edge[0] not in keeping_objects:
                                    keeping_objects.append(keeping_edge[0])
                                if keeping_edge[1] not in keeping_objects:
                                    keeping_objects.append(keeping_edge[1])
                        reverse_mapping = {}
                        distractor_obj_map = {}
                        for i in range(0, len(keeping_objects)):
                            # The following line is way too strict I think!
                            # distractor_size_map[f"$OBJ_{i}"] = int(sampled_world["obj_map"][keeping_objects[i]].size)
                            distractor_obj_pattern_map[f"$OBJ_{i}"] = referent_obj_pattern_map[keeping_objects[i]]
                            distractor_obj_map[f"$OBJ_{i}"] = referent_obj_map[keeping_objects[i]]
                            reverse_mapping[keeping_objects[i]] = f"$OBJ_{i}"

                        distractor_rel_map = OrderedDict({})
                        for keeping_edge in keeping_edges:
                            distractor_rel_map[reverse_mapping[keeping_edge[0]], reverse_mapping[keeping_edge[1]]] = referent_rel_map[keeping_edge]

                        # We need to increment the object counters.
                        distractors_dicts += [{
                                                "grammer_pattern" : self.snap_pattern_to_referent_map(
                                                    distractor_grammer_pattern,
                                                    obj_base_count
                                                ),
                                                "obj_pattern_map" : self.snap_object_map_to_referent_map(
                                                    distractor_obj_pattern_map,
                                                    obj_base_count
                                                ),
                                                "rel_map" : self.snap_relation_map_to_referent_map(
                                                    distractor_rel_map,
                                                    obj_base_count
                                                ),
                                                "obj_map" : self.snap_object_map_to_referent_map(
                                                    distractor_obj_map,
                                                    obj_base_count
                                                ),
                                                "size_map" : self.snap_object_map_to_referent_map(
                                                    distractor_size_map,
                                                    obj_base_count
                                                ),
                                                "distractor_metadata" : distractor_metadata
                                            }]

                        obj_base_count += len(distractor_obj_pattern_map)
                
        elif len(relation_edges) == 0:
            # If there is only a single relation,
            # we will use the replacement method
            # to replace this relation with a new one!
            return []
        elif len(relation_edges) == 1:
            # for this, we simple sample another obj_0?
            distractor_size_map = {}
            distractor_obj_pattern_map = {}
            distractor_obj_map = {}
            distractor_rel_map = OrderedDict({})
            
            distractor_size_map["$OBJ_0"] = int(sampled_world["obj_map"]["$OBJ_0"].size)
            distractor_obj_pattern_map["$OBJ_0"] = referent_obj_pattern_map["$OBJ_0"]
            distractor_obj_map["$OBJ_0"] = referent_obj_map["$OBJ_0"]
            distractor_grammer_pattern = referent_grammer_pattern.split(" ")[0]

            distractor_metadata = {
                "edge" : "null",
                "relation_old_type" : "null", # omit relations
                "full_set" : True,
            }
            
            distractors_dicts += [{
                                    "grammer_pattern" : self.snap_pattern_to_referent_map(
                                        distractor_grammer_pattern,
                                        obj_base_count
                                    ),
                                    "obj_pattern_map" : self.snap_object_map_to_referent_map(
                                        distractor_obj_pattern_map,
                                        obj_base_count
                                    ),
                                    "rel_map" : self.snap_relation_map_to_referent_map(
                                        distractor_rel_map,
                                        obj_base_count
                                    ),
                                    "obj_map" : self.snap_object_map_to_referent_map(
                                        distractor_obj_map,
                                        obj_base_count
                                    ),
                                    "size_map" : self.snap_object_map_to_referent_map(
                                        distractor_size_map,
                                        obj_base_count
                                    ),
                                    "distractor_metadata" : distractor_metadata
                                }]
                
        else:
            if referent_grammer_pattern == "$OBJ_0 ^ $OBJ_1 ^ $OBJ_2":
                # for this, we simple sample another obj_0?
                distractor_size_map = {}
                distractor_obj_pattern_map = {}
                distractor_obj_map = {}
                distractor_rel_map = OrderedDict({})

                # distractor_size_map["$OBJ_0"] = int(sampled_world["obj_map"]["$OBJ_0"].size)
                distractor_obj_pattern_map["$OBJ_0"] = referent_obj_pattern_map["$OBJ_0"]
                distractor_obj_pattern_map["$OBJ_1"] = referent_obj_pattern_map["$OBJ_1"]
                distractor_obj_map["$OBJ_0"] = referent_obj_map["$OBJ_0"]
                distractor_obj_map["$OBJ_1"] = referent_obj_map["$OBJ_1"]
                distractor_grammer_pattern = "$OBJ_0 ^ $OBJ_1"
                distractor_rel_map[("$OBJ_0", "$OBJ_1")] = referent_rel_map[("$OBJ_0", "$OBJ_1")]
                distractor_metadata = {
                    "edge" : ("$OBJ_0", "$OBJ_1"),
                    "relation_old_type" : referent_rel_map[("$OBJ_0", "$OBJ_1")], # omit relations
                    "full_set" : True,
                }

                distractors_dicts += [{
                                        "grammer_pattern" : self.snap_pattern_to_referent_map(
                                            distractor_grammer_pattern,
                                            obj_base_count
                                        ),
                                        "obj_pattern_map" : self.snap_object_map_to_referent_map(
                                            distractor_obj_pattern_map,
                                            obj_base_count
                                        ),
                                        "rel_map" : self.snap_relation_map_to_referent_map(
                                            distractor_rel_map,
                                            obj_base_count
                                        ),
                                        "obj_map" : self.snap_object_map_to_referent_map(
                                            distractor_obj_map,
                                            obj_base_count
                                        ),
                                        "size_map" : self.snap_object_map_to_referent_map(
                                            distractor_size_map,
                                            obj_base_count
                                        ),
                                        "distractor_metadata" : distractor_metadata
                                    }]

            else:
            
                # PR: always fullset.
                # random.shuffle(relation_edges)
                # if full_set:
                #     pass
                # else:
                #    relation_edges = relation_edges[:1] # select only the first element.
                existing_relations = set([v for k, v in referent_rel_map.items()])
                for selected_leaf_edge in relation_edges:

                    distractor_metadata = {
                        "edge" : selected_leaf_edge,
                        "relation_old_type" : referent_rel_map[selected_leaf_edge], # omit relations
                        "full_set" : full_set,
                    }

                    # now for the selected edge, we omit them!
                    distractor_grammer_pattern = referent_grammer_pattern.split(" ")[:-2]
                    distractor_grammer_pattern = " ".join(distractor_grammer_pattern) # omiting the last.

                    distractor_size_map = {}

                    # First, let us make copies.
                    distractor_obj_pattern_map = {}
                    keeping_edges = []
                    keeping_objects = []
                    for keeping_edge in relation_edges:
                        if keeping_edge != selected_leaf_edge:
                            keeping_edges.append(keeping_edge)
                            if keeping_edge[0] not in keeping_objects:
                                keeping_objects.append(keeping_edge[0])
                            if keeping_edge[1] not in keeping_objects:
                                keeping_objects.append(keeping_edge[1])
                    reverse_mapping = {}
                    distractor_obj_map = {}
                    for i in range(0, len(keeping_objects)):
                        # The following line is way too strict I think!
                        # distractor_size_map[f"$OBJ_{i}"] = int(sampled_world["obj_map"][keeping_objects[i]].size)
                        distractor_obj_pattern_map[f"$OBJ_{i}"] = referent_obj_pattern_map[keeping_objects[i]]
                        distractor_obj_map[f"$OBJ_{i}"] = referent_obj_map[keeping_objects[i]]
                        reverse_mapping[keeping_objects[i]] = f"$OBJ_{i}"

                    distractor_rel_map = OrderedDict({})
                    for keeping_edge in keeping_edges:
                        distractor_rel_map[reverse_mapping[keeping_edge[0]], reverse_mapping[keeping_edge[1]]] = referent_rel_map[keeping_edge]

                    # We need to increment the object counters.
                    distractors_dicts += [{
                                            "grammer_pattern" : self.snap_pattern_to_referent_map(
                                                distractor_grammer_pattern,
                                                obj_base_count
                                            ),
                                            "obj_pattern_map" : self.snap_object_map_to_referent_map(
                                                distractor_obj_pattern_map,
                                                obj_base_count
                                            ),
                                            "rel_map" : self.snap_relation_map_to_referent_map(
                                                distractor_rel_map,
                                                obj_base_count
                                            ),
                                            "obj_map" : self.snap_object_map_to_referent_map(
                                                distractor_obj_map,
                                                obj_base_count
                                            ),
                                            "size_map" : self.snap_object_map_to_referent_map(
                                                distractor_size_map,
                                                obj_base_count
                                            ),
                                            "distractor_metadata" : distractor_metadata
                                        }]

                    obj_base_count += len(distractor_obj_pattern_map)

        return distractors_dicts
            
            
    def sample_distractor_grammer_by_relation(
        self, 
        referent_grammer_pattern, 
        referent_obj_pattern_map,
        referent_rel_map,
        referent_obj_map, 
        sampled_world,
        obj_base_count=0,
        full_set=True
    ):
        """
        This will select 1 relation mentioned in the command
        and modify it to a new one. Then, sample distractors
        based on that command (sampling step is outside of 
        this function). This function only construct the semantics
        of distractors.
        """

        distractors_dicts = []
        # We first collect all the relations
        relation_edges = []
        for edge, relation in referent_rel_map.items():
            relation_edges.append(edge)
        random.shuffle(relation_edges)
        if full_set:
            pass
        else:
            relation_edges = relation_edges[:1] # select only the first element.
        
        existing_relations = set([v for k, v in referent_rel_map.items()])
        # print(referent_rel_map)
        for selected_leaf_edge in relation_edges:

            # First, let us make copies.
            distractor_grammer_pattern = copy.deepcopy(referent_grammer_pattern)
            distractor_obj_pattern_map = copy.deepcopy(referent_obj_pattern_map)
            distractor_rel_map = copy.deepcopy(referent_rel_map)
            distractor_obj_map = copy.deepcopy(referent_obj_map)

            # We may need to enforce the size of the distractor due to size descriptors!
            distractor_size_map = {}
        
            selected_surgery = "REL_ADJUST" # Dummy

            distractor_metadata = {
                "edge" : selected_leaf_edge,
                "relation_old_type" : distractor_rel_map[selected_leaf_edge]
            }

            if selected_surgery == "REL_ADJUST":
                # Determine the new relation as not the same one as the current one.
                new_rels = ["$SAME_ROW", "$SAME_COLUMN", "$SAME_SHAPE", "$SAME_COLOR", "$SAME_SIZE", "$IS_INSIDE"]
                new_rels = set(new_rels) - existing_relations # make this very strict!
                # There are something else do not make sense to sample!
                # if "$SIZE" in distractor_obj_pattern_map[selected_leaf_edge[0]]:
                #     new_rels -= set(["$SAME_SIZE"])
                # if "$COLOR" in distractor_obj_pattern_map[selected_leaf_edge[0]]:
                #     new_rels -= set(["$SAME_COLOR"])
                # if "$SHAPE" in distractor_obj_pattern_map[selected_leaf_edge[0]]:
                #    new_rels -= set(["$SAME_SHAPE"])
                new_rel = random.choice(list(new_rels))
                existing_relations.add(new_rel)
                distractor_metadata["relation_new_type"] = new_rel
                distractor_rel_map[selected_leaf_edge] = new_rel
                if new_rel == "$IS_INSIDE":
                    # We can still try to keep the color and size the same.
                    distractor_size_map[selected_leaf_edge[1]] = sampled_world["obj_map"][selected_leaf_edge[1]].size
                    distractor_obj_map[selected_leaf_edge[1]] = sampled_world["obj_map"][selected_leaf_edge[1]].color + " box"
                    distractor_obj_pattern_map[selected_leaf_edge[1]] = '$COLOR $SHAPE'
                else:
                    distractor_size_map[selected_leaf_edge[1]] = sampled_world["obj_map"][selected_leaf_edge[1]].size
                    if "box" in distractor_obj_map[selected_leaf_edge[1]]:
                        # it used to box type object.
                        distractor_size_map[selected_leaf_edge[1]] = sampled_world["obj_map"][selected_leaf_edge[1]].size
                        distractor_obj_map[selected_leaf_edge[1]] = sampled_world["obj_map"][selected_leaf_edge[1]].color + " object"
                        distractor_obj_pattern_map[selected_leaf_edge[1]] = '$COLOR $ABS_SHAPE'
                    else:
                        distractor_size_map[selected_leaf_edge[1]] = sampled_world["obj_map"][selected_leaf_edge[1]].size
                        distractor_obj_map[selected_leaf_edge[1]] = \
                            sampled_world["obj_map"][selected_leaf_edge[1]].color + " " + \
                            sampled_world["obj_map"][selected_leaf_edge[1]].shape
                        distractor_obj_pattern_map[selected_leaf_edge[1]] = '$COLOR $SHAPE'
            else:
                assert False
        
            # We need to increment the object counters.
            distractors_dicts += [{
                                    "grammer_pattern" : self.snap_pattern_to_referent_map(
                                        distractor_grammer_pattern,
                                        obj_base_count
                                    ),
                                    "obj_pattern_map" : self.snap_object_map_to_referent_map(
                                        distractor_obj_pattern_map,
                                        obj_base_count
                                    ),
                                    "rel_map" : self.snap_relation_map_to_referent_map(
                                        distractor_rel_map,
                                        obj_base_count
                                    ),
                                    "obj_map" : self.snap_object_map_to_referent_map(
                                        distractor_obj_map,
                                        obj_base_count
                                    ),
                                    "size_map" : self.snap_object_map_to_referent_map(
                                        distractor_size_map,
                                        obj_base_count
                                    ),
                                    "distractor_metadata" : distractor_metadata
                                }]
            obj_base_count += len(distractor_obj_pattern_map)

        return distractors_dicts

    def sample_distractor_grammer_by_isomorphism(
        self,
        referent_grammer_pattern, 
        referent_obj_pattern_map,
        referent_rel_map,
        referent_obj_map, 
        sampled_world,
        obj_base_count=0,
    ):
        """
        This set of distractors are for learning syntax and grammers.
        If you simply use BoW approach, it will not work because we 
        always instill confusing targets for you with isomorphism of the
        referent graph.

        For example, if the original grounded command is:
        Go to the red square that is inside of the yellow box.

        We can do a isomorphism which is
        Go to the yellow square that is inside of the red box.

        If the model is not understanding the language correctly,
        it will not able to find the referent target correctly.
        """
        # First, let us make copies.
        distractor_grammer_pattern = copy.deepcopy(referent_grammer_pattern)
        distractor_obj_pattern_map = copy.deepcopy(referent_obj_pattern_map)
        distractor_rel_map = copy.deepcopy(referent_rel_map)
        distractor_obj_map = copy.deepcopy(referent_obj_map)
        # We may need to enforce the size of the distractor due to size descriptors!
        distractor_size_map = {}

        shufflable_objects = []
        for obj_name, obj_str in distractor_obj_map.items():
            if obj_name == "$OBJ_0":
                continue # We need to sample distractors of object 0, thus, we keep it intact!
            obj_descriptors = obj_str.split(" ")
            # if this is a box, we don't swap it
            if "box" in obj_descriptors:
                continue
            if "object" in obj_descriptors:
                # "object" itself is not shufflable!
                if len(obj_descriptors) > 1:
                    shufflable_objects.append((obj_name, obj_str))
            else:
                shufflable_objects.append((obj_name, obj_str))
        if len(shufflable_objects) > 2:
            random.shuffle(shufflable_objects)
        shufflable_objects = shufflable_objects[:2]
        
        if len(shufflable_objects) == 1 or len(shufflable_objects) == 0:
            return [] # We simply don't have enough objects to do this.

        # We will shuffle attributes between two objects.
        # We actually shuffle by looking at their relations.
        obj_name_left = shufflable_objects[0][0]
        obj_name_right = shufflable_objects[1][0]
        swap_color = True
        swap_size = False # Let us stop swapping size for now.
        swap_shape = True
        if (obj_name_left, obj_name_right) in distractor_rel_map.keys() or \
            (obj_name_right, obj_name_left) in distractor_rel_map.keys():
            if ((obj_name_left, obj_name_right) in distractor_rel_map.keys() and \
                    distractor_rel_map[(obj_name_left, obj_name_right)] == "SameColor") or \
                ((obj_name_right, obj_name_left) in distractor_rel_map.keys() and \
                     distractor_rel_map[(obj_name_right, obj_name_left)] == "SameColor"):
                swap_color = False
            elif ((obj_name_left, obj_name_right) in distractor_rel_map.keys() and \
                    distractor_rel_map[(obj_name_left, obj_name_right)] == "SameSize") or \
                ((obj_name_right, obj_name_left) in distractor_rel_map.keys() and \
                     distractor_rel_map[(obj_name_right, obj_name_left)] == "SameSize"):
                swap_size = False
            elif ((obj_name_left, obj_name_right) in distractor_rel_map.keys() and \
                    distractor_rel_map[(obj_name_left, obj_name_right)] == "SameShape") or \
                ((obj_name_right, obj_name_left) in distractor_rel_map.keys() and \
                     distractor_rel_map[(obj_name_right, obj_name_left)] == "SameShape"):
                swap_shape = False
            else:
                pass
        
        size_left = self.extract_size(shufflable_objects[0][1])
        size_right = self.extract_size(shufflable_objects[1][1])
        color_left = self.extract_color(shufflable_objects[0][1])
        color_right = self.extract_color(shufflable_objects[1][1])
        shape_left = self.extract_shape(shufflable_objects[0][1])
        shape_right = self.extract_shape(shufflable_objects[1][1])
        
        if size_left == "" and size_right == "":
            swap_size = False
        if color_left == "" and color_right == "":
            swap_color = False
        if shape_left == "" and shape_right == "":
            swap_shape = False
        if shape_left == "box" or shape_right == "box":
            swap_shape = False
        
        if not swap_color and not swap_size and not swap_shape:
            return []
        
        swapping_attribute = []
        if swap_color:
            swapping_attribute += ["color"]
        if swap_size and swap_shape:
            swapping_attribute += ["size+shape"]
        if not swap_size and swap_shape:
            swapping_attribute += ["size+shape"]
        swapping_attribute = random.choice(swapping_attribute)

        left_rebuild = []
        right_rebuild = []
        
        size_shuffled = False
        color_shuffled = False
        shape_shuffled = False
        if swapping_attribute == "color":
            tmp = color_left
            color_left = color_right
            color_right = tmp
            color_shuffled = True
        elif swapping_attribute == "shape":
            tmp = shape_left
            shape_left = shape_right
            shape_right = tmp
            shape_shuffled = True
        elif swapping_attribute == "size+shape":
            tmp = shape_left
            shape_left = shape_right
            shape_right = tmp
            shape_shuffled = True
            
            tmp = size_left
            size_left = size_right
            size_right = tmp
            size_shuffled = True
            
        # We don't swap size!
        if size_left != "":
            left_rebuild.append(size_left)
        if size_right != "":
            right_rebuild.append(size_right)
            
        if color_left != "":
            left_rebuild.append(color_left)
        if color_right != "":
            right_rebuild.append(color_right)

        if shape_left != "":
            left_rebuild.append(shape_left)
        else:
            left_rebuild.append("object")
        if shape_right != "":
            right_rebuild.append(shape_right)
        else:
            right_rebuild.append("object")
        
        if not color_shuffled and not shape_shuffled:
            return []
                
        left_rebuild = " ".join(left_rebuild)
        right_rebuild = " ".join(right_rebuild)
        left_grammer_rebuild = self.convert_object_str_to_grammer(left_rebuild)
        right_grammer_rebuild = self.convert_object_str_to_grammer(right_rebuild)
        
        # It seems like it is possible with our case
        # You need extra cautious of you want to extend for longer logics
        # if left_rebuild == shufflable_objects[1][1] or right_rebuild == shufflable_objects[0][1]:
        #     return [] # we don't allow complete swap!
        
        distractor_obj_pattern_map[obj_name_left] = left_grammer_rebuild 
        distractor_obj_pattern_map[obj_name_right] = right_grammer_rebuild 
        distractor_obj_map[obj_name_left] = left_rebuild
        distractor_obj_map[obj_name_right] = right_rebuild
        
        distractor_metadata = {
            "swapped_pair" : (obj_name_left, obj_name_right),
            "before_pair_obj_str" : (shufflable_objects[0][1], shufflable_objects[1][1]),
            "after_pair_obj_str" : (left_rebuild, right_rebuild),
            "size_shuffled" : size_shuffled,
            "color_shuffled" : color_shuffled,
            "shape_shuffled" : shape_shuffled
        }
        
        return [{
                    "grammer_pattern" : self.snap_pattern_to_referent_map(
                        distractor_grammer_pattern,
                        obj_base_count
                    ),
                    "obj_pattern_map" : self.snap_object_map_to_referent_map(
                        distractor_obj_pattern_map,
                        obj_base_count
                    ),
                    "rel_map" : self.snap_relation_map_to_referent_map(
                        distractor_rel_map,
                        obj_base_count
                    ),
                    "obj_map" : self.snap_object_map_to_referent_map(
                        distractor_obj_map,
                        obj_base_count
                    ),
                    "size_map" : self.snap_object_map_to_referent_map(
                        distractor_size_map,
                        obj_base_count
                    ),
                    "distractor_metadata" : [distractor_metadata]
                }]

    def sample_distractor_grammer_by_attribute(
        self,
        referent_grammer_pattern, 
        referent_obj_pattern_map,
        referent_rel_map,
        referent_obj_map, 
        sampled_world,
        special_shape_size_bound,
        obj_base_count=0,
        full_set=False,
    ):
        """
        We randomly select 1 object and 1 attribute
        that exists in the command to do the attack.
        
        Then, for all objects if size attribute exists
        this function is also responsible for sampling
        dummy size distractors!
        """
        # First, let us make copies.
        distractor_grammer_pattern = copy.deepcopy(referent_grammer_pattern)
        distractor_obj_pattern_map = copy.deepcopy(referent_obj_pattern_map)
        distractor_rel_map = copy.deepcopy(referent_rel_map)
        distractor_obj_map = copy.deepcopy(referent_obj_map)
        
        # Let us do some pre-checks
        # If there are two boxes already
        # Let us stop sampling boxes!
        box_count = 0
        for obj_name, obj in sampled_world['obj_map'].items():
            if obj.shape == "box":
                box_count += 1
        if box_count >= 2:
            full_set = False
        
        # We may need to enforce the size of the distractor due to size descriptors!
        distractor_size_map = OrderedDict({})
        sizing_covered = []
        if full_set:
            obj_pool = []
            for obj_name, obj_grammer in referent_obj_pattern_map.items():
                if "$ABS_SHAPE" in obj_grammer or "box" in distractor_obj_map[obj_name]:
                    continue
                obj_pool += [obj_name]
            # there is a case where, we cannot do anything with this!
            if len(obj_pool) < 1:
                return [] # We simply don't have enough objects to do this.
            obj_selected = random.choice(obj_pool)
            attribute_pool = referent_obj_pattern_map[obj_selected].split(" ")
            attribute_pool = list(set(attribute_pool)-set(["$ABS_SHAPE"]))
            attribute_selected = random.choice(attribute_pool)

            distractor_metadata = {
                "modified_obj" : obj_selected,
                "modified_attribute" : attribute_selected,
            }

            if attribute_selected == "$SIZE":
                sizing_covered.append(obj_selected)
                obj_name = obj_selected
                original_object_str = distractor_obj_map[obj_name]
                obj_grammer = distractor_obj_pattern_map[obj_name]
                original_object = sampled_world['obj_map'][obj_name]
                original_object_size = original_object.size
                if "$COLOR" in obj_grammer:
                    special_shape = \
                        sampled_world['obj_map'][obj_name].color + \
                        " " + sampled_world['obj_map'][obj_name].shape
                else:
                    special_shape = sampled_world['obj_map'][obj_name].shape
                
                if "small" in original_object_str:
                    distractor_size = special_shape_size_bound[special_shape][1]
                elif "big" in original_object_str:
                    distractor_size = special_shape_size_bound[special_shape][0]
                distractor_size_map[obj_name] = distractor_size
                distractor_shape = original_object.shape
                tmp_name = ""
                if "$COLOR" in obj_grammer:
                    distractor_color = original_object.color
                    new_object_grammer = "$SIZE $COLOR $SHAPE" # $SIZE is a must right?
                    tmp_name = distractor_color + " " + distractor_shape
                else:
                    distractor_color = self.object_vocabulary.sample_color()
                    new_object_grammer = "$SIZE $SHAPE"
                    tmp_name = distractor_shape
                if "small" in original_object_str:
                    tmp_name = "big" + " " + tmp_name
                elif "big" in original_object_str:
                    tmp_name = "small" + " " + tmp_name
                else:
                    pass # Not Implemented
                distractor_obj_map[obj_name] = tmp_name
                distractor_obj_pattern_map[obj_name] = new_object_grammer

                # Then, we will also consider other object sizes. Basically,
                # we keep them the same, unless they form SameShape relation
                # with our core object.
                for _obj_name, _obj in sampled_world['obj_map'].items():
                    if _obj_name != obj_name:
                        if (_obj_name, obj_name) in referent_rel_map and \
                            referent_rel_map[(_obj_name, obj_name)] == "SameSize":
                            distractor_size_map[_obj_name] = distractor_size
                        elif (obj_name, _obj_name) in referent_rel_map and \
                            referent_rel_map[(obj_name, _obj_name)] == "SameSize":
                            distractor_size_map[_obj_name] = distractor_size
                        else:
                            distractor_size_map[_obj_name] = _obj.size
            elif attribute_selected == "$COLOR":
                original_object_name = obj_selected
                original_object_str = distractor_obj_map[original_object_name]
                original_object = sampled_world['obj_map'][original_object_name]
                new_color = self.object_vocabulary.sample_color(_exclude=[original_object.color])
                new_object_str = new_color + " " + original_object.shape
                new_object_grammer = "$COLOR $SHAPE"
                distractor_obj_map[original_object_name] = new_object_str
                distractor_obj_pattern_map[original_object_name] = new_object_grammer
            elif attribute_selected == "$SHAPE":

                original_object_name = obj_selected
                original_object_str = distractor_obj_map[original_object_name]
                original_object = sampled_world['obj_map'][original_object_name]
                new_shape = self.object_vocabulary.sample_shape(_exclude=[original_object.shape])
                new_object_str = original_object.color + " " + new_shape
                new_object_grammer = "$COLOR $SHAPE"
                distractor_obj_map[original_object_name] = new_object_str
                distractor_obj_pattern_map[original_object_name] = new_object_grammer
                
            # Now for all other objects with size attribute, we need
            # to ground them even if it is not relational.
            base_distractor_count = len(list(distractor_obj_map.keys()))
            for obj_name, obj_grammer in referent_obj_pattern_map.items():
                if "$SIZE" in obj_grammer:
                    if obj_name not in sizing_covered:
                        # Just sample a single one.
                        new_obj_name = f"OBJ_{base_distractor_count}"
                        original_object_str = referent_obj_map[obj_name]

                        tmp_name = " ".join(original_object_str.split(" ")[1:])
                        if "small" in original_object_str:
                            tmp_name = "big" + " " + tmp_name
                        elif "big" in original_object_str:
                            tmp_name = "small" + " " + tmp_name
                        else:
                            pass # Not Implemented

                        if "$COLOR" in obj_grammer:
                            special_shape = \
                                sampled_world['obj_map'][obj_name].color + \
                                " " + sampled_world['obj_map'][obj_name].shape
                        else:
                            special_shape = sampled_world['obj_map'][obj_name].shape
                        if "small" in original_object_str:
                            distractor_size = special_shape_size_bound[special_shape][1]
                        elif "big" in original_object_str:
                            distractor_size = special_shape_size_bound[special_shape][0]
                        # the above size if the proposed size!

                        # Let us iterate through the map, if there is
                        # already a shape working as the size counterparts
                        # we don't need it!
                        color = self.extract_color(original_object_str)
                        shape = self.extract_shape(original_object_str)
                        if color != "" and shape != "object":
                            found = False
                            for obj_str, obj in sampled_world['obj_map'].items():
                                if obj.color == color and obj.shape == shape:
                                    if obj.size == distractor_size:
                                        found = True
                                        break
                            if found:
                                continue
                        elif color != "" and shape == "object":
                            found = False
                            for obj_str, obj in sampled_world['obj_map'].items():
                                if obj.color == color:
                                    if obj.size == distractor_size:
                                        found = True
                                        break
                            if found:
                                continue
                        elif color == "" and shape == "object":
                            found = False
                            for obj_str, obj in sampled_world['obj_map'].items():
                                if obj.size == distractor_size:
                                    found = True
                                    break
                            if found:
                                continue
                        elif color == "" and shape != "object":
                            found = False
                            for obj_str, obj in sampled_world['obj_map'].items():
                                if obj.shape == shape:
                                    if obj.size == distractor_size:
                                        found = True
                                        break
                            if found:
                                continue

                        distractor_obj_map[new_obj_name] = tmp_name
                        distractor_obj_pattern_map[new_obj_name] = obj_grammer
                        distractor_size_map[new_obj_name] = distractor_size
                        base_distractor_count += 1
                
        else:
            # We cleanup, and simply place random objects.
            distractor_grammer_pattern = "DUMMY"
            distractor_obj_pattern_map.clear()
            distractor_rel_map.clear()
            distractor_obj_map.clear()
            
            distractor_metadata = {
                "modified_obj" : None,
                "modified_attribute" : None,
            }
            # Now for all other objects with size attribute, we need
            # to ground them even if it is not relational.
            base_distractor_count = len(list(distractor_obj_map.keys()))
            for obj_name, obj_grammer in referent_obj_pattern_map.items():
                if "$SIZE" in obj_grammer:
                    if obj_name not in sizing_covered:
                        # Just sample a single one.
                        new_obj_name = f"OBJ_{base_distractor_count}"
                        original_object_str = referent_obj_map[obj_name]
                        
                        tmp_name = " ".join(original_object_str.split(" ")[1:])
                        if "small" in original_object_str:
                            tmp_name = "big" + " " + tmp_name
                        elif "big" in original_object_str:
                            tmp_name = "small" + " " + tmp_name
                        else:
                            pass # Not Implemented

                        if "$COLOR" in obj_grammer:
                            special_shape = \
                                sampled_world['obj_map'][obj_name].color + \
                                " " + sampled_world['obj_map'][obj_name].shape
                        else:
                            special_shape = sampled_world['obj_map'][obj_name].shape
                        
                        # We need to be a little careful when
                        # dealing with abstract shape object
                        # for example, big object -> small object.
                            
                        if "small" in original_object_str:
                            distractor_size = special_shape_size_bound[special_shape][1]
                        elif "big" in original_object_str:
                            distractor_size = special_shape_size_bound[special_shape][0]
                        # the above size if the proposed size!
                        
                        # Let us iterate through the map, if there is
                        # already a shape working as the size counterparts
                        # we don't need it!
                        color = self.extract_color(original_object_str)
                        shape = self.extract_shape(original_object_str)
                        if color != "" and shape != "object":
                            found = False
                            for obj_str, obj in sampled_world['obj_map'].items():
                                if obj.color == color and obj.shape == shape:
                                    if obj.size == distractor_size:
                                        found = True
                                        break
                            if found:
                                continue
                        elif color != "" and shape == "object":
                            found = False
                            for obj_str, obj in sampled_world['obj_map'].items():
                                if obj.color == color:
                                    if obj.size == distractor_size:
                                        found = True
                                        break
                            if found:
                                continue
                        elif color == "" and shape == "object":
                            found = False
                            for obj_str, obj in sampled_world['obj_map'].items():
                                if obj.size == distractor_size:
                                    found = True
                                    break
                            if found:
                                continue
                        elif color == "" and shape != "object":
                            found = False
                            for obj_str, obj in sampled_world['obj_map'].items():
                                if obj.shape == shape:
                                    if obj.size == distractor_size:
                                        found = True
                                        break
                            if found:
                                continue
                            
                        distractor_obj_map[new_obj_name] = tmp_name
                        distractor_obj_pattern_map[new_obj_name] = obj_grammer
                        distractor_size_map[new_obj_name] = distractor_size
                        base_distractor_count += 1
        return [{
            "grammer_pattern" : self.snap_pattern_to_referent_map(
                distractor_grammer_pattern,
                obj_base_count
            ),
            "obj_pattern_map" : self.snap_object_map_to_referent_map(
                distractor_obj_pattern_map,
                obj_base_count
            ),
            "rel_map" : self.snap_relation_map_to_referent_map(
                distractor_rel_map,
                obj_base_count
            ),
            "obj_map" : self.snap_object_map_to_referent_map(
                distractor_obj_map,
                obj_base_count
            ),
            "size_map" : self.snap_object_map_to_referent_map(
                distractor_size_map,
                obj_base_count
            ),
            "distractor_metadata" : [distractor_metadata]
        }]

In [22]:
# test out the vocabulary
intransitive_verbs = ["walk"]
transitive_verbs = ["push", "pull"]
adverbs = ["while zigzagging", "while spinning", "cautiously", "hesitantly"]
nouns = ["circle", "cylinder", "square", "box"]
color_adjectives = ["red", "blue", "green", "yellow"]
size_adjectives = ["big", "small"]
relative_pronouns = ["that is"]
relation_clauses = ["in the same row as", 
                    "in the same column as", 
                    "in the same color as", 
                    "in the same shape as", 
                    "in the same size as",
                    "inside of"]
vocabulary = Vocabulary.initialize(intransitive_verbs=intransitive_verbs,
                                   transitive_verbs=transitive_verbs, adverbs=adverbs, nouns=nouns,
                                   color_adjectives=color_adjectives,
                                   size_adjectives=size_adjectives, 
                                   relative_pronouns=relative_pronouns, 
                                   relation_clauses=relation_clauses)

min_object_size = 1
max_object_size = 4
object_vocabulary = ObjectVocabulary(shapes=vocabulary.get_semantic_shapes(),
                                     colors=vocabulary.get_semantic_colors(),
                                     min_size=min_object_size, max_size=max_object_size)

grammer = Grammer(vocabulary)

simulator = Simulator(
    object_vocabulary, vocabulary, 
    grid_size=6, 
    n_object_max=13,
)

In [8]:
# Let us explore different distractor sampling!
pattern = ""
if pattern == "p4":
    ReaSCAN_data_file = f"ReaSCAN-compositional{pattern}/data-train.txt"
else:
    ReaSCAN_data_file = f"ReaSCAN-compositional{pattern}/data-compositional-splits.txt"
ReaSCAN_data_json = json.load(open(os.path.join("../../data-files-updated/", ReaSCAN_data_file)))

In [23]:
synthetic_tasks = []

per_command_world_target_count = 3 # Need to increase this for some tasks!
per_command_world_retry_max = 200
sampled_command_count = 5000

examples = ReaSCAN_data_json["examples"]["train"]
random.shuffle(examples)
sampled_examples = random.sample(examples, k=sampled_command_count)
progress = 0

for example_selected in sampled_examples:
    
    if example_selected['grammer_pattern'] != '$OBJ_0 ^ $OBJ_1 & $OBJ_2':
        continue
    
    progress += 1
    if progress % 10 == 0:
        print(f"count={len(synthetic_tasks)}")
        print(f"progress={progress}")
    if len(synthetic_tasks) > 10000:
        break # we get enough!
        
    rel_map = OrderedDict({})
    for ele in example_selected["relation_map"]:
        rel_map[tuple(ele[0])] = ele[1]
    example_struct = {'obj_pattern_map': example_selected["object_pattern_map"],
     'rel_map': rel_map,
     'obj_map': example_selected["object_expression"],
     'grammer_pattern': example_selected['grammer_pattern'],
     'adverb': example_selected['adverb_in_command'],
     'verb': example_selected['verb_in_command']}
    
    obj_pattern_map = example_struct["obj_pattern_map"]
    rel_map = example_struct["rel_map"]
    obj_map = example_struct["obj_map"]
    grammer_pattern = example_struct["grammer_pattern"]
    verb = example_struct["verb"]
    adverb = example_struct["adverb"]

    for _ in range(per_command_world_target_count):
        for i in range(per_command_world_retry_max): # retry essentially!
            sampled_world = simulator.sample_situations_from_grounded_grammer(
                    copy.deepcopy(grammer_pattern), 
                    copy.deepcopy(obj_pattern_map), 
                    copy.deepcopy(rel_map), 
                    copy.deepcopy(obj_map),
                    is_plot=False,
                    include_relation_distractor=True, 
                    include_attribute_distractor=False, 
                    include_isomorphism_distractor=False, 
                    include_random_distractor=True,
                    full_relation_probability=1.0, # 0.5 seems to work as well!
                    debug=False
                )
            
            # print(sampled_world)
            # _ = simulator._world.render_simple()
            
            assert len(sampled_world['obj_map']) == len(simulator._world.get_current_situation().to_representation()["placed_objects"])

            graph = ReaSCANGraph(
                objects=sampled_world["obj_map"], 
                object_patterns=sampled_world["obj_pattern_map"], 
                vocabulary=vocabulary,
                positions=sampled_world["pos_map"], 
                referred_object=sampled_world["referred_obj"],
                debug=False
            )

            pattern_graph = ReaSCANGraph(
                objects=obj_map, 
                object_patterns=None,
                vocabulary=vocabulary,
                relations=rel_map, 
                referred_object='$OBJ_0', 
                debug=False
            )

            potential_referent_target = graph.find_referred_object_super_fast(
                pattern_graph, referred_object='$OBJ_0', 
                debug=False
            )

            if len(potential_referent_target) == 1 and '$OBJ_0' in potential_referent_target:
                # test_unique_find += 1
                # print(f"{test_unique_find} / {i+1} unique solution find!")

                # Form the command with grounded determiners!
                obj_determiner_map = graph.find_determiners(
                    pattern_graph, 
                    referred_object='$OBJ_0', 
                    debug=False,
                )
                command_str = grammer.repre_str_command(
                    grammer_pattern, rel_map, obj_map, 
                    obj_determiner_map, 
                    verb,
                    adverb,
                )

                # Form the golden label for the action list!
                is_transitive = False
                if verb in simulator.vocabulary.get_transitive_verbs():
                    is_transitive = True
                # Direct walk.
                action = "walk" # this is definit!
                primitive_command = simulator.vocabulary.translate_word(action)
                target_position = sampled_world["situation"].target_object.position
                simulator._world.go_to_position(
                    position=target_position, manner=adverb, 
                    primitive_command=primitive_command
                )
                # Object actions.
                if is_transitive:
                    semantic_action = simulator.vocabulary.translate_word(verb)
                    simulator._world.move_object_to_wall(action=semantic_action, manner=adverb)
                target_commands, _ = simulator._world.get_current_observations()

                has_relation_distractor = False
                full_relation_distractor = True
                for rel_bool in sampled_world["distractor_switch_map"]["relation"]:
                    if rel_bool:
                        has_relation_distractor = True
                    else:
                        full_relation_distractor = False

                # Save all relevant information for a task.
                task_struct = OrderedDict({
                    "command": ",".join(command_str.split(" ")),
                    "grammer_pattern": grammer_pattern,
                    "meaning": ",".join(command_str.split(" ")),
                    "derivation": grammer_pattern,
                    "situation": sampled_world["situation"].to_representation(),
                    "target_commands": ",".join(target_commands),
                    "verb_in_command": verb,
                    "adverb_in_command": adverb,
                    "referred_target": obj_map["$OBJ_0"],
                    "object_pattern_map": obj_pattern_map,
                    "relation_map": [(k, v) for k, v in rel_map.items()],
                    "object_expression": obj_map,
                    "n_object": len(sampled_world["obj_map"]),
                    "n_distractor": len(sampled_world["obj_map"])-len(obj_map),
                    "full_relation_distractor": full_relation_distractor,
                    "has_relation_distractor": has_relation_distractor,
                    "has_attribute_distractor": sampled_world["distractor_switch_map"]["attribute"],
                    "has_isomorphism_distractor": sampled_world["distractor_switch_map"]["isomorphism"],
                    "has_random_distractor": True if sampled_world["n_random_distractor"] != -1 else False,
                    "n_random_distractor": sampled_world["n_random_distractor"] if sampled_world["n_random_distractor"] != -1 else 0,
                    "relation_distractor_metadata": sampled_world["relation_distractor_metadata"],
                    "attribute_distractor_metadata": sampled_world["attribute_distractor_metadata"],
                    "isomorphism_distractor_metadata": sampled_world["isomorphism_distractor_metadata"],
                    "random_distractor_metadata": sampled_world["random_distractor_metadata"],
                })
                synthetic_tasks += [task_struct]
                break


count=27
progress=10
count=57
progress=20
count=87
progress=30
count=117
progress=40
count=147
progress=50
count=177
progress=60
count=207
progress=70
count=237
progress=80
count=267
progress=90
count=297
progress=100
count=327
progress=110


KeyboardInterrupt: 

In [24]:
synthetic_tasks = {
    "test" : synthetic_tasks
}
dataset_representation = {
    "grid_size": 6,
    "type_grammar": "ReaSCAN-Grammer",
    "min_object_size": 1,
    "max_object_size": 4,
    "percentage_train": 0.0,
    "examples": synthetic_tasks,
    "intransitive_verbs": intransitive_verbs,
    "transitive_verbs": transitive_verbs,
    "adverbs": adverbs,
    "nouns": nouns,
    "color_adjectives": color_adjectives,
    "size_adjectives": size_adjectives,
    "relative_pronouns": relative_pronouns,
    "relation_clauses": relation_clauses,
}

In [25]:
split_name = "-f1"
with open(f"../../data-files-updated/ReaSCAN-compositional{split_name}/data-compositional-splits.txt", "w") as fd:
    json.dump(dataset_representation, fd, indent=4)

Unseen co-occurence of relations and objects but with seen atomic concepts

In [4]:
# Currently, we hard-code the pattern!
grammer_pattern = '$OBJ_0 ^ $OBJ_1 & $OBJ_2'
logger.info(f"Including pattern:= {grammer_pattern}...")
# Sampling relations
relations = grammer.sample_object_relation_grammer(
    '$OBJ_0', 
    grammer.build_dependency_graph(grammer_pattern))
command_structs = {}
for relation in relations:
    obj_pattern_map = relation[0]
    rel_map = relation[1]
    grammer_bindings = grammer.grounding_grammer_with_vocabulary(grammer_pattern, obj_pattern_map, rel_map)
    for obj_map in grammer_bindings:
        # here, we also sample the verb and adverb bindings!
        adverb_enhance_list = vocabulary.get_adverbs()
        adverb_enhance_list += [""]
        command_struct = OrderedDict({
            "obj_pattern_map" : obj_pattern_map,
            "rel_map" : rel_map,
            "obj_map" : obj_map,
            "grammer_pattern" : grammer_pattern,
            "adverb" : random.choice(adverb_enhance_list),
            "verb" : random.choice(vocabulary.get_transitive_verbs() + vocabulary.get_intransitive_verbs()),
        })
        command_str = grammer.repre_str_command(
            grammer_pattern, rel_map, obj_map, 
            {"$OBJ_0" : "the", "$OBJ_1" : "a", "$OBJ_2" : "a"}, 
            command_struct["verb"],
            command_struct["adverb"],
        )
        command_structs[command_str] = command_struct

2021-06-14 12:20 Including pattern:= $OBJ_0 ^ $OBJ_1 & $OBJ_2...


In [5]:
seen_command_structs = {}
seen_concepts = {} # add in seen concepts, so we can select concepts that are seen, but new composites!
seen_object_co = set([])
seen_rel_co = set([])
seen_rel_obj_co = set([])

for example_selected in ReaSCAN_data_json["examples"]["train"]:
    rel_map = OrderedDict({})
    for ele in example_selected["relation_map"]:
        rel_map[tuple(ele[0])] = ele[1]
    example_struct = OrderedDict({
        'obj_pattern_map': example_selected["object_pattern_map"],
        'rel_map': rel_map,
        'obj_map': example_selected["object_expression"],
        'grammer_pattern': example_selected['grammer_pattern'],
        'adverb': example_selected['adverb_in_command'],
        'verb': example_selected['verb_in_command']
    })
    obj_co = []
    for k, v in example_selected["object_expression"].items():
        if v not in seen_concepts:
            seen_concepts[v] = 1
        else:
            seen_concepts[v] += 1
        obj_co += [v]
    obj_co.sort()
    seen_object_co.add(tuple(obj_co))
    
    rel_co = []
    for k, v in rel_map.items():
        if v not in seen_concepts:
            seen_concepts[v] = 1
        else:
            seen_concepts[v] += 1
        rel_co += [v]
    rel_co.sort()
    seen_rel_co.add(tuple(rel_co))
    
    if example_selected['grammer_pattern'] == "$OBJ_0 ^ $OBJ_1 & $OBJ_2":
        pair = (rel_map[("$OBJ_0", "$OBJ_1")], example_selected["object_expression"]["$OBJ_1"])
        seen_rel_obj_co.add(pair)
        pair = (rel_map[("$OBJ_0", "$OBJ_2")], example_selected["object_expression"]["$OBJ_2"])
        seen_rel_obj_co.add(pair)
    elif example_selected['grammer_pattern'] == "$OBJ_0 ^ $OBJ_1":
        pair = (rel_map[("$OBJ_0", "$OBJ_1")], example_selected["object_expression"]["$OBJ_1"])
        seen_rel_obj_co.add(pair)

    # if example_selected['grammer_pattern'] == "$OBJ_0 ^ $OBJ_1 & $OBJ_2":
    command_str = grammer.repre_str_command(
        example_selected['grammer_pattern'], rel_map, example_selected["object_expression"], 
        {"$OBJ_0" : "the", "$OBJ_1" : "a", "$OBJ_2" : "a"}, 
        example_selected['verb_in_command'],
        example_selected['adverb_in_command'],
    )
    seen_command_structs[command_str] = example_struct

In [6]:
unseen_obj_co_command_structs = []
unseen_obj_rel_co_command_structs = []
unseen_rel_co_command_structs = []
unseen_obj_rel_pair_command_structs = []
for k, v in command_structs.items():

    if k not in seen_command_structs:

        # we need to ensure concepts are seen before though!
        concept_seen = True
        obj_co = []
        for kk, vv in v['obj_map'].items():
            if vv not in seen_concepts.keys():
                concept_seen = False
                break
            obj_co += [vv]
        
        rel_co = []
        for kk, vv in v['rel_map'].items():
            if vv not in seen_concepts.keys():
                concept_seen = False
                break
            rel_co += [vv]

        pair = (v['rel_map'][("$OBJ_0", "$OBJ_1")], v["obj_map"]["$OBJ_1"])
        rel_obj_co_1 = pair
        pair = (v['rel_map'][("$OBJ_0", "$OBJ_2")], v["obj_map"]["$OBJ_2"])
        rel_obj_co_2 = pair

        if concept_seen:
            obj_co.sort()
            obj_co = tuple(obj_co)
            rel_co.sort()
            rel_co = tuple(rel_co)
            if rel_co not in seen_rel_co and obj_co in seen_object_co:
                unseen_rel_co_command_structs += [v]
            if obj_co not in seen_object_co and rel_co in seen_rel_co:
                unseen_obj_co_command_structs += [v]
            if obj_co not in seen_object_co and rel_co not in seen_rel_co:
                unseen_obj_rel_co_command_structs += [v]

In [7]:
synthetic_tasks = []
progress = 0

per_command_world_target_count = 3 # Need to increase this for some tasks!
per_command_world_retry_max = 200
sampled_command_count = 5000

for example_selected in unseen_obj_co_command_structs:

    progress += 1
    if progress % 50 == 0:
        print(f"count={len(synthetic_tasks)}")
        print(f"progress={progress}")
    if len(synthetic_tasks) > 10000:
        break # we get enough!

    obj_pattern_map = example_selected["obj_pattern_map"]
    rel_map = example_selected["rel_map"]
    obj_map = example_selected["obj_map"]
    grammer_pattern = example_selected["grammer_pattern"]
    verb = example_selected["verb"]
    adverb = example_selected["adverb"]

    for _ in range(per_command_world_target_count):
        for i in range(per_command_world_retry_max): # retry essentially!
            sampled_world = simulator.sample_situations_from_grounded_grammer(
                    copy.deepcopy(grammer_pattern), 
                    copy.deepcopy(obj_pattern_map), 
                    copy.deepcopy(rel_map), 
                    copy.deepcopy(obj_map),
                    is_plot=False,
                    include_relation_distractor=True, 
                    include_attribute_distractor=True, 
                    include_isomorphism_distractor=True, 
                    include_random_distractor=True,
                    full_relation_probability=1.0, # 0.5 seems to work as well!
                    debug=False
                )

            assert len(sampled_world['obj_map']) == len(simulator._world.get_current_situation().to_representation()["placed_objects"])

            graph = ReaSCANGraph(
                objects=sampled_world["obj_map"], 
                object_patterns=sampled_world["obj_pattern_map"], 
                vocabulary=vocabulary,
                positions=sampled_world["pos_map"], 
                referred_object=sampled_world["referred_obj"],
                debug=False
            )

            pattern_graph = ReaSCANGraph(
                objects=obj_map, 
                object_patterns=None,
                vocabulary=vocabulary,
                relations=rel_map, 
                referred_object='$OBJ_0', 
                debug=False
            )

            potential_referent_target = graph.find_referred_object_super_fast(
                pattern_graph, referred_object='$OBJ_0', 
                debug=False
            )

            if len(potential_referent_target) == 1 and '$OBJ_0' in potential_referent_target:
                # print(f"{test_unique_find} / {i+1} unique solution find!")
                # Form the command with grounded determiners!
                obj_determiner_map = graph.find_determiners(
                    pattern_graph, 
                    referred_object='$OBJ_0', 
                    debug=False,
                )
                command_str = grammer.repre_str_command(
                    grammer_pattern, rel_map, obj_map, 
                    obj_determiner_map, 
                    verb,
                    adverb,
                )

                # Form the golden label for the action list!
                is_transitive = False
                if verb in simulator.vocabulary.get_transitive_verbs():
                    is_transitive = True
                # Direct walk.
                action = "walk" # this is definit!
                primitive_command = simulator.vocabulary.translate_word(action)
                target_position = sampled_world["situation"].target_object.position
                simulator._world.go_to_position(
                    position=target_position, manner=adverb, 
                    primitive_command=primitive_command
                )
                # Object actions.
                if is_transitive:
                    semantic_action = simulator.vocabulary.translate_word(verb)
                    simulator._world.move_object_to_wall(action=semantic_action, manner=adverb)
                target_commands, _ = simulator._world.get_current_observations()

                has_relation_distractor = False
                full_relation_distractor = True
                for rel_bool in sampled_world["distractor_switch_map"]["relation"]:
                    if rel_bool:
                        has_relation_distractor = True
                    else:
                        full_relation_distractor = False

                # Save all relevant information for a task.
                task_struct = OrderedDict({
                    "command": ",".join(command_str.split(" ")),
                    "grammer_pattern": grammer_pattern,
                    "meaning": ",".join(command_str.split(" ")),
                    "derivation": grammer_pattern,
                    "situation": sampled_world["situation"].to_representation(),
                    "target_commands": ",".join(target_commands),
                    "verb_in_command": verb,
                    "adverb_in_command": adverb,
                    "referred_target": obj_map["$OBJ_0"],
                    "object_pattern_map": obj_pattern_map,
                    "relation_map": [(k, v) for k, v in rel_map.items()],
                    "object_expression": obj_map,
                    "n_object": len(sampled_world["obj_map"]),
                    "n_distractor": len(sampled_world["obj_map"])-len(obj_map),
                    "full_relation_distractor": full_relation_distractor,
                    "has_relation_distractor": has_relation_distractor,
                    "has_attribute_distractor": sampled_world["distractor_switch_map"]["attribute"],
                    "has_isomorphism_distractor": sampled_world["distractor_switch_map"]["isomorphism"],
                    "has_random_distractor": True if sampled_world["n_random_distractor"] != -1 else False,
                    "n_random_distractor": sampled_world["n_random_distractor"] if sampled_world["n_random_distractor"] != -1 else 0,
                    "relation_distractor_metadata": sampled_world["relation_distractor_metadata"],
                    "attribute_distractor_metadata": sampled_world["attribute_distractor_metadata"],
                    "isomorphism_distractor_metadata": sampled_world["isomorphism_distractor_metadata"],
                    "random_distractor_metadata": sampled_world["random_distractor_metadata"],
                })
                synthetic_tasks += [task_struct]
                break

count=147
progress=50
count=297
progress=100
count=447
progress=150
count=597
progress=200
count=747
progress=250
count=897
progress=300
count=1047
progress=350
count=1197
progress=400
count=1347
progress=450
count=1497
progress=500
count=1647
progress=550
count=1797
progress=600
count=1947
progress=650
count=2097
progress=700
count=2247
progress=750
count=2397
progress=800
count=2547
progress=850
count=2697
progress=900
count=2847
progress=950
count=2997
progress=1000
count=3147
progress=1050
count=3297
progress=1100
count=3447
progress=1150
count=3597
progress=1200
count=3747
progress=1250
count=3897
progress=1300
count=4047
progress=1350
count=4197
progress=1400
count=4308
progress=1450
count=4407
progress=1500
count=4533
progress=1550
count=4677
progress=1600
count=4811
progress=1650
count=4950
progress=1700
count=5088
progress=1750
count=5229
progress=1800
count=5379
progress=1850
count=5529
progress=1900
count=5676
progress=1950
count=5813
progress=2000
count=5952
progress=2050
c

In [8]:
synthetic_tasks = {
    "test" : synthetic_tasks
}
dataset_representation = {
    "grid_size": 6,
    "type_grammar": "ReaSCAN-Grammer",
    "min_object_size": 1,
    "max_object_size": 4,
    "percentage_train": 0.0,
    "examples": synthetic_tasks,
    "intransitive_verbs": intransitive_verbs,
    "transitive_verbs": transitive_verbs,
    "adverbs": adverbs,
    "nouns": nouns,
    "color_adjectives": color_adjectives,
    "size_adjectives": size_adjectives,
    "relative_pronouns": relative_pronouns,
    "relation_clauses": relation_clauses,
}

In [9]:
split_name = "-d2"
with open(f"../../data-files-updated/ReaSCAN-compositional{split_name}/data-compositional-splits.txt", "w") as fd:
    json.dump(dataset_representation, fd, indent=4)

Two "that is" clauses

In [10]:
possible_command_structs = []
for example_selected in ReaSCAN_data_json["examples"]["train"]:
    rel_map = OrderedDict({})
    for ele in example_selected["relation_map"]:
        rel_map[tuple(ele[0])] = ele[1]
    example_struct = OrderedDict({
        'obj_pattern_map': example_selected["object_pattern_map"],
        'rel_map': rel_map,
        'obj_map': example_selected["object_expression"],
        'grammer_pattern': example_selected['grammer_pattern'], # force it to be double recursive
        'adverb': example_selected['adverb_in_command'],
        'verb': example_selected['verb_in_command']
    })
    
    # the second object cannot be box!
    if example_struct['grammer_pattern'] == '$OBJ_0 ^ $OBJ_1 & $OBJ_2':
        if "box" not in example_struct["obj_map"]['$OBJ_1']:
            
            # other filters as well!
            if rel_map[("$OBJ_0", "$OBJ_1")] in ["$SAME_ROW", "$SAME_COLUMN"] and \
                rel_map[("$OBJ_0", "$OBJ_2")] in ["$SAME_ROW", "$SAME_COLUMN"]:
                example_struct['grammer_pattern'] = '$OBJ_0 ^ $OBJ_1 ^ $OBJ_2'
                rel_map_new = OrderedDict({})
                rel_map_new[("$OBJ_0", "$OBJ_1")] = example_struct['rel_map'][("$OBJ_0", "$OBJ_1")]
                rel_map_new[("$OBJ_1", "$OBJ_2")] = example_struct['rel_map'][("$OBJ_0", "$OBJ_2")]
                example_struct['rel_map'] = rel_map_new
                possible_command_structs += [example_struct]

In [11]:
synthetic_tasks = []
progress = 0
for example_selected in possible_command_structs:

    progress += 1
    if progress % 10 == 0:
        print(f"count={len(synthetic_tasks)}")
        print(f"progress={progress}")
    if len(synthetic_tasks) > 8000:
        break # we get enough!
        
    obj_pattern_map = example_selected["obj_pattern_map"]
    rel_map = example_selected["rel_map"]
    obj_map = example_selected["obj_map"]
    grammer_pattern = example_selected["grammer_pattern"]
    verb = example_selected["verb"]
    adverb = example_selected["adverb"]

    for _ in range(per_command_world_target_count):
        for i in range(per_command_world_retry_max): # retry essentially!
            sampled_world = simulator.sample_situations_from_grounded_grammer(
                    copy.deepcopy(grammer_pattern), 
                    copy.deepcopy(obj_pattern_map), 
                    copy.deepcopy(rel_map), 
                    copy.deepcopy(obj_map),
                    is_plot=False,
                    include_relation_distractor=True, 
                    include_attribute_distractor=True, 
                    include_isomorphism_distractor=True, 
                    include_random_distractor=True,
                    full_relation_probability=1.0, # 0.5 seems to work as well!
                    debug=False
                )

            assert len(sampled_world['obj_map']) == len(simulator._world.get_current_situation().to_representation()["placed_objects"])

            graph = ReaSCANGraph(
                objects=sampled_world["obj_map"], 
                object_patterns=sampled_world["obj_pattern_map"], 
                vocabulary=vocabulary,
                positions=sampled_world["pos_map"], 
                referred_object=sampled_world["referred_obj"],
                debug=False
            )

            pattern_graph = ReaSCANGraph(
                objects=obj_map, 
                object_patterns=None,
                vocabulary=vocabulary,
                relations=rel_map, 
                referred_object='$OBJ_0', 
                debug=False
            )

            potential_referent_target = graph.find_referred_object_super_fast(
                pattern_graph, referred_object='$OBJ_0', 
                pattern=grammer_pattern,
                debug=False
            )

            if len(potential_referent_target) == 1 and '$OBJ_0' in potential_referent_target:
                # print(f"{test_unique_find} / {i+1} unique solution find!")
                # Form the command with grounded determiners!
                obj_determiner_map = graph.find_determiners(
                    pattern_graph, 
                    referred_object='$OBJ_0', 
                    debug=False,
                )
                command_str = grammer.repre_str_command(
                    grammer_pattern, rel_map, obj_map, 
                    obj_determiner_map, 
                    verb,
                    adverb,
                )

                # Form the golden label for the action list!
                is_transitive = False
                if verb in simulator.vocabulary.get_transitive_verbs():
                    is_transitive = True
                # Direct walk.
                action = "walk" # this is definit!
                primitive_command = simulator.vocabulary.translate_word(action)
                target_position = sampled_world["situation"].target_object.position
                simulator._world.go_to_position(
                    position=target_position, manner=adverb, 
                    primitive_command=primitive_command
                )
                # Object actions.
                if is_transitive:
                    semantic_action = simulator.vocabulary.translate_word(verb)
                    simulator._world.move_object_to_wall(action=semantic_action, manner=adverb)
                target_commands, _ = simulator._world.get_current_observations()

                has_relation_distractor = False
                full_relation_distractor = True
                for rel_bool in sampled_world["distractor_switch_map"]["relation"]:
                    if rel_bool:
                        has_relation_distractor = True
                    else:
                        full_relation_distractor = False

                # Save all relevant information for a task.
                task_struct = OrderedDict({
                    "command": ",".join(command_str.split(" ")),
                    "grammer_pattern": grammer_pattern,
                    "meaning": ",".join(command_str.split(" ")),
                    "derivation": grammer_pattern,
                    "situation": sampled_world["situation"].to_representation(),
                    "target_commands": ",".join(target_commands),
                    "verb_in_command": verb,
                    "adverb_in_command": adverb,
                    "referred_target": obj_map["$OBJ_0"],
                    "object_pattern_map": obj_pattern_map,
                    "relation_map": [(k, v) for k, v in rel_map.items()],
                    "object_expression": obj_map,
                    "n_object": len(sampled_world["obj_map"]),
                    "n_distractor": len(sampled_world["obj_map"])-len(obj_map),
                    "full_relation_distractor": full_relation_distractor,
                    "has_relation_distractor": has_relation_distractor,
                    "has_attribute_distractor": sampled_world["distractor_switch_map"]["attribute"],
                    "has_isomorphism_distractor": sampled_world["distractor_switch_map"]["isomorphism"],
                    "has_random_distractor": True if sampled_world["n_random_distractor"] != -1 else False,
                    "n_random_distractor": sampled_world["n_random_distractor"] if sampled_world["n_random_distractor"] != -1 else 0,
                    "relation_distractor_metadata": sampled_world["relation_distractor_metadata"],
                    "attribute_distractor_metadata": sampled_world["attribute_distractor_metadata"],
                    "isomorphism_distractor_metadata": sampled_world["isomorphism_distractor_metadata"],
                    "random_distractor_metadata": sampled_world["random_distractor_metadata"],
                })
                synthetic_tasks += [task_struct]
                break

count=27
progress=10
count=57
progress=20
count=87
progress=30
count=116
progress=40
count=146
progress=50
count=172
progress=60
count=200
progress=70
count=230
progress=80
count=259
progress=90
count=287
progress=100
count=317
progress=110
count=347
progress=120
count=377
progress=130
count=407
progress=140
count=435
progress=150
count=465
progress=160
count=490
progress=170
count=520
progress=180
count=549
progress=190
count=579
progress=200
count=607
progress=210
count=637
progress=220
count=667
progress=230
count=696
progress=240
count=726
progress=250
count=756
progress=260
count=785
progress=270
count=814
progress=280
count=842
progress=290
count=872
progress=300
count=902
progress=310
count=932
progress=320
count=960
progress=330
count=990
progress=340
count=1020
progress=350
count=1050
progress=360
count=1076
progress=370
count=1106
progress=380
count=1136
progress=390
count=1166
progress=400
count=1196
progress=410
count=1226
progress=420
count=1253
progress=430
count=1283
pro

In [12]:
synthetic_tasks = {
    "test" : synthetic_tasks
}
dataset_representation = {
    "grid_size": 6,
    "type_grammar": "ReaSCAN-Grammer",
    "min_object_size": 1,
    "max_object_size": 4,
    "percentage_train": 0.0,
    "examples": synthetic_tasks,
    "intransitive_verbs": intransitive_verbs,
    "transitive_verbs": transitive_verbs,
    "adverbs": adverbs,
    "nouns": nouns,
    "color_adjectives": color_adjectives,
    "size_adjectives": size_adjectives,
    "relative_pronouns": relative_pronouns,
    "relation_clauses": relation_clauses,
}

In [13]:
split_name = "-e"
with open(f"../../data-files-updated/ReaSCAN-compositional{split_name}/data-compositional-splits.txt", "w") as fd:
    json.dump(dataset_representation, fd, indent=4)